In [1]:
import pandas as pd
import glob
import os
import numpy as np
import filepath
import send_email 
import pygsheets
import infrastructure 


localfolder=filepath.output_folder

#import publisher from google sheet 
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
rxmgref = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Tzda6Djr3zQmOhWu7Ief3GVR9Cjaml8238CeX7chj_U/edit#gid=1620368362') 
publisher_raw  = rxmgref.worksheet('title','Publisher Configurations').get_as_df()
publisher_raw.to_csv(localfolder+'smartsheet/Publisher.csv')
publisher = publisher_raw[['NEW DP.DS or DP.sV','PUBID']].drop_duplicates()

# import SMS OMS
offer_sheet = infrastructure.get_smartsheet('offers_sms')

## import La Nina. 
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
lanina = gc.open_by_url('https://docs.google.com/spreadsheets/d/1obszkCQoE0ELOR1O0CrLVETUEmEIWlGuyAmK3FgWSJg/edit#gid=1099746391') 
lanina_sheet  = lanina.worksheet('title','La Nina (Current)').get_as_df()
lanina_sheet.to_csv(localfolder+'smartsheet/La_Nina.csv')

## EMIT 
emit = infrastructure.get_smartsheet('emit')
email_pubid = emit['Revenue Pub ID'].astype(str).str.split('.',expand = True)[0].unique().tolist()

jobs = pd.read_csv(localfolder + 'SMS_SC_SS_Jobs.csv', dtype={'job_id' :'Int64'})
creative_stats = pd.read_csv(localfolder + 'SMS_SC_SS_CreativesStats.csv', dtype={'CreativeId' :'str'})
raw_creative_stats = creative_stats
offers = pd.read_csv(localfolder + 'SMS_SC_SS_Offers.csv', usecols = ['name','hitpath_offer_id', 'type','status','redirect_type','conversion_event','conversion_payout','currency'] )
historic_data = pd.read_csv(localfolder + 'SS_data.csv')#  data before Nov1
flows = pd.read_csv(localfolder + 'SMS_SC_SS_Flows.csv')

# add some information in flows
flows['Revenue Source'] = 'Short Code - SS Flow'
flows['Code_Type'] = 'Short Code' 
flows['Shortcode Name'] = flows['Name'].str.split('-',expand = True)[1]
flows['Shortcode1'] = flows['Shortcode'].str.extract('(\d{5})')
flows = flows.drop('Shortcode',axis = 1)
flows['Dataset'] = flows['Name'].str.split('-',expand = True)[0]
flows['Dataset'] = flows['Dataset'].str.replace('JM.ONP','JET.ONP')
flows['Dataset'] = flows['Dataset'].str.replace('JM.NTC','JET.NTC')
flows = flows.rename(columns=({'CountDeliver':'Delivered', 'CostDeliver':'Cost','CountClick':'Clicks','Shortcode1':'Shortcode', 'CountUnsub':'Optout'}))
flows['Date'] = pd.to_datetime(flows['Period'].str[:10])
flows['AR Flow ID']=flows['Id'].astype('str')
flows['AR Day'] = flows['OfferName'].str.extract('(\d+)\s*\(')
flows.loc[flows['AR Day'].isna(),'AR Day'] =  flows['OfferName'].str.extract('AR(\d)')[0]
flows.loc[flows['AR Day'].isna(),'AR Day'] = 'Null'
flows['Hitpath ID'] = flows['OfferName'].str.extract(r'(\d{4,5})')
flows['AR Flow'] = flows['AR Flow ID'] + '_Day_' + flows['Shortcode Name']+'_'+  flows['AR Day']
flows_clean = flows[['Hitpath ID','Date','Dataset','Shortcode','Shortcode Name','Revenue Source','Code_Type','Delivered','Optout','Cost','Clicks','AR Flow','AR Day','AR Flow ID']]
flows_clean = flows_clean.merge(publisher[['NEW DP.DS or DP.sV','PUBID']], left_on ='Dataset', right_on = 'NEW DP.DS or DP.sV', how = 'left' )
flows_clean = flows_clean.rename(columns=({'PUBID':'Affiliate_id'}))
flows_clean.loc[flows_clean['Shortcode Name'] == 'FLC', 'Shortcode'] = '51797'
flows_clean.loc[flows_clean['Shortcode Name'] == 'CSS', 'Shortcode'] = '70610'
flows_clean.loc[flows_clean['Shortcode Name'] == 'HZB', 'Shortcode'] = '44345'
flows_clean.loc[flows_clean['Shortcode Name'] == 'MBC', 'Shortcode'] = '80837'
flows_clean.loc[flows_clean['Shortcode'] == '51797', 'Shortcode Name'] = 'FLC'
flows_clean.loc[flows_clean['Shortcode'] == '70610', 'Shortcode Name'] = 'CSS'
flows_clean.loc[flows_clean['Shortcode'] == '44345', 'Shortcode Name'] = 'HZB'
flows_clean.loc[flows_clean['Shortcode'] == '80837', 'Shortcode Name'] = 'MBC'

# change some segment name 
creative_stats['Segment'] = creative_stats['Segment'].str.replace('WW.YFA','WWM.YFA')
creative_stats['Segment'] = creative_stats['Segment'].str.replace('ARM.CR','AI.CC')
creative_stats['Segment'] = creative_stats['Segment'].str.replace('RH.3CS','RHD.CC')
creative_stats['Segment'] = creative_stats['Segment'].str.replace('ED.247L','EDM.247L')
creative_stats['Segment'] = creative_stats['Segment'].str.replace('PA.PS','PA.SWP')
creative_stats['Segment'] = creative_stats['Segment'].str.replace('FM.YS','FSM.YS')
creative_stats['Segment'] = creative_stats['Segment'].str.replace('CM_','CM.OSR_')
creative_stats['Segment'] = creative_stats['Segment'].str.replace('CM.OSR.OSR','CM.OSR')
creative_stats['Segment'] = creative_stats['Segment'].str.replace('CM.OSR.OSR.OSR','CM.OSR')
creative_stats = creative_stats.rename(columns=({'DeliverCount':'Delivered', 'TotalCost':'Cost','ClickCount':'Clicks','UnsubCount':'Optout'}))
creative_id_na = creative_stats.loc[creative_stats['CreativeId'].isna()]
jobs_optout = jobs[['id','optout']]
creative_stats =  creative_stats.merge(jobs_optout, left_on = 'JobId' ,right_on='id', how='left')
jobid_count = creative_stats['JobId'].value_counts().to_dict()
creative_stats['Optout'] = creative_stats.apply(lambda row: row['optout'] / jobid_count[row['JobId']], axis=1)
creative_stats = creative_stats.groupby(['JobId', 'Tstamp', 'Offer', 'Segment', 'CreativeId', 'CreativeName','Creative']).sum().reset_index()
print(creative_stats['Delivered'].sum())
jobs['segments'] = jobs['segments'].str.replace('WW.YFA','WWM.YFA')
jobs['segments'] = jobs['segments'].str.replace('ARM.CR','AI.CC')
jobs['segments'] = jobs['segments'].str.replace('RH.3CS','RHD.CC')
jobs['segments'] = jobs['segments'].str.replace('ED.247L','EDM.247L')
jobs['segments'] = jobs['segments'].str.replace('PA.PS','PA.SWP')
jobs['segments'] = jobs['segments'].str.replace('FM.YS','FSM.YS')
jobs['segments'] = jobs['segments'].str.replace('CM_','CM.OSR_')
jobs['segments'] = jobs['segments'].str.replace('CM.OSR.OSR','CM.OSR')
jobs['segments'] = jobs['segments'].str.replace('CM.OSR.OSR.OSR','CM.OSR')


#clean jobs file
#drop columns with nulls
jobs.isna().sum()
drop_columns_jobs = ['remote_status']
jobs.drop(columns=drop_columns_jobs, inplace=True, axis=1)
jobs = jobs.rename(columns={ 'id':'job_id', 'name':'job_name'})
#convert date fields to correct format
jobs['start_tstamp'] = pd.to_datetime(jobs['start_tstamp'])
jobs['end_tstamp'] = pd.to_datetime(jobs['end_tstamp'])
jobs['scheduled_tstamp'] = pd.to_datetime(jobs['scheduled_tstamp'],format = 'mixed')
jobs['Scheduling Time'] = jobs['scheduled_tstamp'].dt.tz_convert('US/Pacific').dt.strftime('%Y-%m-%d %H:%M')
creative_stats =  creative_stats.merge(jobs[['job_id','Scheduling Time']], left_on = 'JobId' ,right_on='job_id', how='left')
creative_id_na =  creative_id_na.merge(jobs[['job_id','Scheduling Time']], left_on = 'JobId' ,right_on='job_id', how='left')

#clean creative_stats
creative_stats.isna().sum()
creative_stats['Tstamp'] = pd.to_datetime(creative_stats['Tstamp'])

#clean offers
offers.isna().sum()
offers = offers.rename(columns={ 'id':'offer_id', 'name':'offer'})

#####engagement stats for all sends#####
#merge above df with reveneu file to get all of reveneu information
## combine all months revenue CSVs into one master-revenue file.
os.chdir(localfolder + "SMS Rev")

all_files = [i for i in glob.glob('*.{}'.format('csv'))]
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_files ])
####Exporting master revenue file####
combined_csv.to_csv( localfolder + "SMS_master_revenue.csv", index=False, encoding='utf-8-sig')


combined_csv = pd.read_csv(localfolder + 'SMS_master_revenue.csv', dtype={'advertiser_name':'str','campaign_name':'str'})
daily_revenue = combined_csv
daily_revenue['date'] = pd.to_datetime(daily_revenue['date']).dt.date
daily_revenue_gpby = daily_revenue.groupby(['affiliate_id','date'])['amount'].sum().reset_index()
jump_page_csv = pd.read_csv( localfolder+"jumppage_click.csv")
jumppage_clicks = jump_page_csv.rename(columns = {'subid_1':"subid_2"})
print(combined_csv['amount'].sum())
combined_csv = pd.concat([combined_csv,jumppage_clicks])
print(combined_csv['amount'].sum())
print(combined_csv['Jump Page Clicks'].sum())
#check subids
subs = ["subid_1","subid_2", "subid_5"]
def sid(d):
    sid = "subid_3"
    for j in subs:
        if d[f"{j} uc"]>=5:
            sid = j
    return d[sid]
for i in subs:
    combined_csv[i] = combined_csv[i].astype(str)
    combined_csv[f"{i} uc"] = combined_csv[i].str.count("_")
#merge all subids from 1,2 and 5 in subid
combined_csv["sub_id"] = combined_csv.apply(sid,axis=1)

########## SS BEGIN #########
combined_csv['first_split']=combined_csv['sub_id'].str.split('_').str[0]
""" 
jumppage_clicks["sub_id"] = jumppage_clicks["subid_1"]
subs = ["sub_id"]
# doing same cleaning with jumpapge files  
for i in subs:
    jumppage_clicks[i] = jumppage_clicks[i].astype(str)
    jumppage_clicks[f"{i} uc"] = jumppage_clicks[i].str.count("_")
#merge all subids from 1,2 and 5 in subid

jumppage_clicks['first_split']=jumppage_clicks['sub_id'].str.split('_').str[0]
"""
# get the date based on subid date 
combined_csv = combined_csv.reset_index(drop=True)
combined_csv['Dash_Date_from_subid'] = combined_csv['sub_id'].str.extract(r'(\d{1,2}[A-Za-z]{3}\d{2})')
combined_csv['Dash_Date_from_subid'] = pd.to_datetime(combined_csv['Dash_Date_from_subid'], format='%d%b%y',errors='coerce')
combined_csv['date'] = pd.to_datetime(combined_csv['date'],errors='coerce')
combined_csv.loc[combined_csv['Dash_Date_from_subid'].isna(),'Dash_Date_from_subid'] = combined_csv['date']
combined_csv['date'] = combined_csv['Dash_Date_from_subid']
combined_csv.loc[combined_csv['campaign_id']==8202]

/opt/anaconda3/lib/python3.8/site-packages/pygsheets/worksheet.py:1477: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')


127149385


/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_5409/263324719.py:127: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_csv = pd.concat([pd.read_csv(f) for f in all_files ])
/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_5409/2633

5617932.581938998
5617932.581938998
6548036.0


,date,advertiser_name,campaign_id,campaign_name,site_id,affiliate_id,affiliate_name,subid_1,subid_2,subid_3,...,revenue,plcategory_name,plsubcategory_name,Jump Page Clicks,subid_1 uc,subid_2 uc,subid_5 uc,sub_id,first_split,Dash_Date_from_subid
2563016,2021-06-30,Progrexion,8202.0,Lexington Law Linkout - 33824 - Progrexion (SMS),460917.0,460917,1 - SMS LM: Jet Marketing,460917,MP_FT_460917_CT4231_JM.OP_C15_6571_82162_30Jun...,NaN,...,NaN,SMS,LM,NaN,0,10,0,MP_FT_460917_CT4231_JM.OP_C15_6571_82162_30Jun...,MP,2021-06-30
2574054,2021-06-24,Progrexion,8202.0,Lexington Law Linkout - 33824 - Progrexion (SMS),460939.0,460939,1 - SMS LM: Aramis_Credit,460939,03.04.21_ARM_460939_a_6571_SC_MP_j3-l_||postid||,NaN,...,NaN,SMS,LM,NaN,0,8,0,03.04.21_ARM_460939_a_6571_SC_MP_j3-l_||postid||,03.04.21,2021-06-24
2599482,2021-06-15,Progrexion,8202.0,Lexington Law Linkout - 33824 - Progrexion (SMS),460654.0,460654,1 - SMS: RentOwnClub,460654,MP_HZB_460654_CT4073_I.RC_C45_6269_44345_15Jun...,NaN,...,NaN,SMS,RentOwnClub,NaN,0,13,0,MP_HZB_460654_CT4073_I.RC_C45_6269_44345_15Jun...,MP,2021-06-15
2616897,2021-06-06,Progrexion,8202.0,Lexington Law Linkout - 33824 - Progrexion (SMS),460654.0,460654,1 - SMS: RentOwnClub,460654,MP_HZB_460654_CT3983_I.RC_C15_6264_44345_6Jun2...,NaN,...,NaN,SMS,RentOwnClub,NaN,0,13,0,MP_HZB_460654_CT3983_I.RC_C15_6264_44345_6Jun2...,MP,2021-06-06
3975880,2022-11-02,NaN,8202.0,NaN,NaN,460654,NaN,nan,SMS_SS_SC_HZB_44345_460654_416732_6264_124652_...,NaN,...,NaN,NaN,NaN,4.0,0,10,0,SMS_SS_SC_HZB_44345_460654_416732_6264_124652_...,SMS,2022-11-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5875352,2023-09-02,NaN,8202.0,NaN,NaN,461680,NaN,nan,SS_SC_MBC_80837_461680_AR0_11468_514234_2Sep23...,NaN,...,NaN,NaN,NaN,1.0,0,9,0,SS_SC_MBC_80837_461680_AR0_11468_514234_2Sep23...,SS,2023-09-02
5875353,2023-08-30,NaN,8202.0,NaN,NaN,461680,NaN,nan,SS_SC_MBC_80837_461680_AR0_11468_514234_30Aug2...,NaN,...,NaN,NaN,NaN,1.0,0,9,0,SS_SC_MBC_80837_461680_AR0_11468_514234_30Aug2...,SS,2023-08-30
5875354,2023-08-31,NaN,8202.0,NaN,NaN,461680,NaN,nan,SS_SC_MBC_80837_461680_AR0_11468_514234_31Aug2...,NaN,...,NaN,NaN,NaN,1.0,0,9,0,SS_SC_MBC_80837_461680_AR0_11468_514234_31Aug2...,SS,2023-08-31
5875355,2023-09-03,NaN,8202.0,NaN,NaN,461680,NaN,nan,SS_SC_MBC_80837_461680_AR0_11468_514234_3Sep23...,NaN,...,NaN,NaN,NaN,1.0,0,9,0,SS_SC_MBC_80837_461680_AR0_11468_514234_3Sep23...,SS,2023-09-03


In [2]:
# Import Long Code data from MD

lc = gc.open_by_url('https://docs.google.com/spreadsheets/d/1De9UzMw5POUuS90s7dMKsYNTQVnxTj1dvAvwMhOCC5U/edit#gid=0') 
lc_wk_df  = lc.worksheet('title','Long Code').get_as_df()
lc_wk_df['SMS Cost'] = lc_wk_df['SMS Cost'].str.split('$',expand = True)[1].astype('float')
lc_wk_df = lc_wk_df.rename(columns = {'Offer':'Sub_Id'})
lc_rev = combined_csv.loc[(combined_csv['first_split']=='MD') | (combined_csv['first_split']=='TB') ]
#lc_rev = combined_csv.loc[(combined_csv['first_split']=='MD') ]
lc_rev.loc[lc_rev['Dash_Date_from_subid'].isna(), 'Dash_Date_from_subid'] = lc_rev['date']
lc_rev =lc_rev.rename(columns = {"Dash_Date_from_subid":'Date'})
#  define the pattern
pattern = r'(.+\d{1,2}[A-Za-z]{3}\d{2})'
# extract the pattern
lc_rev['Sub_Id'] = lc_rev['sub_id'].str.extract(pattern)
lc_rev_summary = lc_rev.groupby(['Sub_Id','Date'])['amount'].sum().reset_index()
lc_df = pd.concat([lc_wk_df,lc_rev_summary],axis = 0, ignore_index=True)
lc_df = lc_df.groupby(['Sub_Id','Date']).sum().reset_index()  
lc_df = lc_df.fillna(0)
                           
                           
lc_df.loc[lc_df['Sub_Id'] == '01JUN23_8838_W4_EDU_SVT','Sub_Id'] = 'MD_LC_OMG_SVT_460918__8838_OT_01JUN23'
lc_df.loc[lc_df['Sub_Id'] == '05Jun23_MD_LC_OMG_SVT_460918_W1_11714_T1','Sub_Id'] = 'MD_LC_OMG_SVT_460918__11714_OT_05Jun23'
lc_df.loc[lc_df['Sub_Id'] == '11714-2_OMG_SVT','Sub_Id'] = 'MD_LC_OMG_SVT_460918__11714_OT_'
lc_df.loc[lc_df['Sub_Id'] == 'MD_LC_OMG_SVT_460919_00066_9088__29Jul23','Sub_Id'] = 'MD_LC_OMG_SVT_460919_00066_9088_P_29Jul23'
lc_df.loc[lc_df['Sub_Id'] == 'MD_LC_OMG_SVT_460920_00068_12076__30Jul23','Sub_Id']  = 'MD_LC_OMG_SVT_460920_00068_12076_P_30Jul23'
lc_df['split_column'] = lc_df['Sub_Id'].str.split('_')
lc_df['subid_uc'] = lc_df['Sub_Id'].str.count('_')
lc_df['Date'] = lc_df['split_column'].str[8]
lc_df['Date']= pd.to_datetime(lc_df['Date'], format='%d%b%y',errors='coerce')
lc_df['Send Strategy'] = lc_df['split_column'].str[7]
lc_df['Hitpath_Offer_ID'] = lc_df['split_column'].str[6]
lc_df['Affiliate_Id'] = lc_df['split_column'].str[4]
lc_df['Long Code Content ID'] = lc_df['split_column'].str[5]
lc_df['Shortcode Name'] = ''
lc_df.loc[lc_df['split_column'].str[3] == 'SVT','Shortcode Name' ] = 'SVT'
lc_df.loc[lc_df['split_column'].str[3] == 'UAA','Shortcode Name' ] = 'UAA'
lc_df['Send Strategy'] = lc_df['Send Strategy'].str.replace('T0','OT')
lc_df['Send Strategy'] = lc_df['Send Strategy'].str.replace('T1','OT') 
lc_df.loc[lc_df['split_column'].str[0] == 'TB', 'Revenue Source'] = 'Long Code - Textback'
lc_df.loc[lc_df['split_column'].str[0] == 'MD', 'Revenue Source'] = 'Long Code - Mobile Drips'
lc_df.loc[lc_df['split_column'].str[0] == 'TB', 'Send Strategy'] = 'P'
lc_df.loc[lc_df['split_column'].str[0] == 'TB', 'Long Code Content ID'] = np.nan

# currently we don't consider split test content option 
lc_df['amount'] = lc_df['amount'].fillna(0)
print(lc_df.loc[lc_df['split_column'].str[0] == 'MD',]['amount'].sum())
lc_lanina = lanina_sheet[(lanina_sheet['Long Code Content ID'].isna() == False) & ((lanina_sheet['Long Code Content ID'] !=""))]
lc_lanina['Long Code Content ID'] = lc_lanina['Long Code Content ID'].astype(str).str.zfill(5)
lc_df_full = lc_df.merge(lc_lanina[['Long Code Content ID','Reporting Content ID','Content']],copy = False, how= 'left', on = 'Long Code Content ID')
print(lc_df_full.loc[lc_df_full['split_column'].str[0] == 'MD',]['amount'].sum())
lc_df_full['Code_Type'] = 'Long Code'
lc_df_full_11 = lc_df_full[lc_df_full['Date'] >= '2022-11-01']

lc_df_full = lc_df_full.rename(columns = {'Qty':'Sent','Daily Success Qty':'Delivered','Fail Qty': 'Undelivered',\
                            'Clicks Qty':'Clicks','SMS Cost':'Cost',\
                            'amount':'Revenue','Long Code Content ID':'Creative_Id', 'Reporting Content ID':'Creativename','Content':'Creative'})

lc_df_full = lc_df_full[['Date','Affiliate_Id', 'Hitpath_Offer_ID','Sent','Delivered','Undelivered','Clicks',\
       'Cost','Revenue', 'Send Strategy', 'Shortcode Name', 'Revenue Source', 'Code_Type','Sub_Id','Creative_Id','Creativename','Creative']]


""" 
merged_data = merged_data[['Date','Scheduling Time', 'Offer','Hitpath_Offer_ID','DP.SV','Affiliate_Id', 'DP&Pub','Job_Id', 'Job_Name','Creative_Id','Creativename','Creative','Send Strategy', 'Shortcode', 'Start_Tstamp', 'Segments', 'Revenue','Jump Page Clicks', 'Delivered', 'Not_Delivered', 'Optout', 'Clicks',
       'Cost', 'Ecpm', 'Time', 'Publisher', 'Campaign', 'Route',  'Carrier', 'Dataset', 'Message',
       'Responder Template', 'Keyword', 'Responder', 'Router Domain Name' , 'c1', 'Responded', 'Response Rate', 'CTR',
        'Gross Profit' , 'Gross Margin', 'RPU' ,'Provider', 'Code_Type','Revenue Source',
     'Ar Day','Sub_Id','Campaign_Id','Roi','Shortcode Name','Total']]
""" 

274.45000000000005
274.45000000000005


/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_5409/2988448243.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lc_lanina['Long Code Content ID'] = lc_lanina['Long Code Content ID'].astype(str).str.zfill(5)


" \nmerged_data = merged_data[['Date','Scheduling Time', 'Offer','Hitpath_Offer_ID','DP.SV','Affiliate_Id', 'DP&Pub','Job_Id', 'Job_Name','Creative_Id','Creativename','Creative','Send Strategy', 'Shortcode', 'Start_Tstamp', 'Segments', 'Revenue','Jump Page Clicks', 'Delivered', 'Not_Delivered', 'Optout', 'Clicks',\n       'Cost', 'Ecpm', 'Time', 'Publisher', 'Campaign', 'Route',  'Carrier', 'Dataset', 'Message',\n       'Responder Template', 'Keyword', 'Responder', 'Router Domain Name' , 'c1', 'Responded', 'Response Rate', 'CTR',\n        'Gross Profit' , 'Gross Margin', 'RPU' ,'Provider', 'Code_Type','Revenue Source',\n     'Ar Day','Sub_Id','Campaign_Id','Roi','Shortcode Name','Total']]\n"

In [4]:
#flows_clean.to_csv('/Users/liliguo/Desktop/offer wall performance/creative_forflow.csv')

In [5]:
combined_csv['date'] = pd.to_datetime(combined_csv['date'])
combined_csv_ss_only = combined_csv[(combined_csv['first_split']=='SS')  |  (combined_csv['first_split']=='SMS')] 
combined_csv_ss_only = combined_csv_ss_only[combined_csv_ss_only['date']>='2022-11-01'].reset_index(drop=True)
print(combined_csv_ss_only['amount'].sum())
#combined_csv_ss_only= combined_csv_ss_only.drop_duplicates()
#count number of underscores in subid
combined_csv_ss_only['subid_uc'] = combined_csv_ss_only.sub_id.str.count('_')
#combined_csv_ss_only['subid_uc']=combined_csv.sub_id.str.count('_')

#ignoring the subids that  are not formatted correctly ex: SS_HZB_{{datasource_id}}_{{job_id}}_ALL.SMS_10434_SM_44345_{{today_d}}{{today_mon}}{{today_yy}}_1_{{member_id}}
#they have very less revenue under them
#these are with sub id uc <12
combined_csv_ss_only = combined_csv_ss_only[combined_csv_ss_only['subid_uc'] < 12]

combined_csv_ss_only.loc[combined_csv_ss_only['advertiser_name'] == 'NIC','campaign_id'] = 10267
Offer_name = combined_csv_ss_only.groupby(['campaign_id','campaign_name']).count().reset_index()[['campaign_id','campaign_name']]

# get jump page sum and revenue sum 
combined_csv_ss_only['split_column'] = combined_csv_ss_only['sub_id'].str.split('_')
combined_csv_ss_only.loc[(combined_csv_ss_only['first_split'] == 'SS')  , 'offer_id'] = combined_csv_ss_only['split_column'].str[6]
combined_csv_ss_only.loc[(combined_csv_ss_only['first_split'] == 'SMS')  , 'offer_id'] = combined_csv_ss_only['split_column'].str[7]
combined_csv_ss_only.loc[(combined_csv_ss_only['offer_id'].str.isdigit() == False), 'offer_id'] =  combined_csv_ss_only['split_column'].str[5]
combined_csv_ss_only.loc[(combined_csv_ss_only['offer_id'].str.isdigit() == False), 'offer_id'] =  combined_csv_ss_only['split_column'].str[4]
combined_csv_ss_only['offer_id'] = combined_csv_ss_only['offer_id'].astype('str').str.split('.',expand = True)[0]
combined_csv_ss_only['campaign_id']= combined_csv_ss_only['campaign_id'].astype('str').str.split('.',expand = True)[0]
combined_csv_ss_only.loc[(combined_csv_ss_only['campaign_id'].isna()) | (combined_csv_ss_only['campaign_id'] == 'nan'),'campaign_id' ] = combined_csv_ss_only['offer_id']
combined_csv_ss_only['Offer Type'] = 'Single Offer'
combined_csv_ss_only.loc[(combined_csv_ss_only['campaign_id'] != combined_csv_ss_only['offer_id']), 'Offer Type' ] = 'Offer Wall'
offerwall = combined_csv_ss_only.loc[combined_csv_ss_only['Offer Type']=='Offer Wall',]
combined_csv_ss_only = combined_csv_ss_only[['date', 'campaign_id',
       'affiliate_id',  'amount', 'Jump Page Clicks',
        'sub_id', 'first_split','Dash_Date_from_subid', 'subid_uc','Offer Type']]
combined_csv_ss_only['amount'] = combined_csv_ss_only['amount'].fillna(0)
combined_csv_ss_only[ 'Jump Page Clicks'] = combined_csv_ss_only[ 'Jump Page Clicks'].fillna(0)
combined_csv_ss_only = combined_csv_ss_only.groupby(['date', 'campaign_id', 'affiliate_id', 'sub_id', 'first_split','Dash_Date_from_subid', 'subid_uc','Offer Type'])[[ 'amount', 'Jump Page Clicks']].sum().reset_index()
# use for verification 
jumppageclicks1 = combined_csv_ss_only['Jump Page Clicks'].sum()
revenue1 = combined_csv_ss_only['amount'].sum()

1195254.3580549995


In [6]:
#*** Extracting creative information. ****#
# find creative id from list of creative ids from creativestats file.
combined_csv_ss_only['split_column'] = combined_csv_ss_only['sub_id'].str.split('_')
combined_csv_ss_only.loc[combined_csv_ss_only[combined_csv_ss_only['first_split'] == 'SS'].index , 'creative_id'] = combined_csv_ss_only['split_column'].str[7]
combined_csv_ss_only.loc[combined_csv_ss_only[combined_csv_ss_only['first_split'] == 'SMS'].index , 'creative_id'] = combined_csv_ss_only['split_column'].str[8]
combined_csv_ss_only.loc[combined_csv_ss_only[combined_csv_ss_only['first_split'] == 'SS'].index , 'Job_id'] = combined_csv_ss_only['split_column'].str[5]
combined_csv_ss_only.loc[combined_csv_ss_only[combined_csv_ss_only['first_split'] == 'SMS'].index , 'Job_id'] = combined_csv_ss_only['split_column'].str[6]
combined_csv_ss_only.loc[combined_csv_ss_only[combined_csv_ss_only['first_split'] == 'SS'].index , 'Shortcode'] = combined_csv_ss_only['split_column'].str[3]
combined_csv_ss_only.loc[combined_csv_ss_only[combined_csv_ss_only['first_split'] == 'SMS'].index , 'Shortcode'] = combined_csv_ss_only['split_column'].str[4] 


# some AR subids in december formatted incorrectly, some residuals formatted incorrectly.
#creative ids are length>6, creative ids len =1 are ARs, ignoring both the cases and repalcing creative ids with nans for rest of the length values
combined_csv_ss_only['creative_idlen'] = combined_csv_ss_only['creative_id'].str.len()
combined_csv_ss_only.loc[combined_csv_ss_only[~combined_csv_ss_only['creative_idlen'].isin([1,6])].index,'creative_id']=np.nan 

# shortcode is valid only when it's a 5-digit number
combined_csv_ss_only.loc[combined_csv_ss_only['Shortcode'] == 'FLC', 'Shortcode'] = '51797'
combined_csv_ss_only['shortcode_idlen'] = combined_csv_ss_only['Shortcode'].str.len()
combined_csv_ss_only.loc[combined_csv_ss_only['shortcode_idlen'] != 5,'Shortcode']=np.nan 

#reformat jobs from residuals and AR
combined_csv_ss_only['job_idlen'] = combined_csv_ss_only['Job_id'].str.len()
combined_csv_ss_only.loc[combined_csv_ss_only[~combined_csv_ss_only['job_idlen'].isin([1,6])].index,'Job_id']=np.nan 
combined_csv_ss_only_reformatjobs = combined_csv_ss_only[combined_csv_ss_only['Job_id'].isna() ]
jobs['job_id'] =jobs['job_id'].astype('str')
"""
for i in jobs['job_id']:
    check_len = combined_csv_ss_only_reformatjobs[combined_csv_ss_only_reformatjobs['sub_id'].str.contains(i)]
    if len(check_len) > 0:
        combined_csv_ss_only_reformatjobs.loc[check_len.index,'Job_id'] = i
"""
# Extract the job IDs to a set for faster membership testing
creative_stats['JobId'] = creative_stats['JobId'].astype('str')
job_ids = set(jobs['job_id']) | set(creative_stats['JobId']) - set(['0'])
#creativeid = set(creative_stats['Creative_Id'])
#combined_csv_ss_only['creative_id'] = combined_csv_ss_only_reformatjobs['sub_id'].str.extract(f'({"|".join(creativeid)})')

# Use str.extract to extract the job ID from the sub_id column
combined_csv_ss_only_reformatjobs['Job_id'] = combined_csv_ss_only_reformatjobs['sub_id'].str.extract(f'({"|".join(job_ids)})')

combined_csv_ss_only =   pd.concat([combined_csv_ss_only_reformatjobs,(combined_csv_ss_only[~combined_csv_ss_only['Job_id'].isna()])] )

combined_csv_ss_only = combined_csv_ss_only[combined_csv_ss_only['creative_idlen']!= 8]
combined_csv_ss_only = combined_csv_ss_only[~combined_csv_ss_only['creative_idlen'].isna()]
#get all creatives information.
combined_csv_ss_only['Job_id'] = combined_csv_ss_only['Job_id'].astype('float')
combined_csv_ss_only['creative_id'] = combined_csv_ss_only['creative_id'].replace('6Oct22', np.nan)
combined_csv_ss_only['creative_id'] = combined_csv_ss_only['creative_id'].astype('float')

creative_stats['CreativeId'] = creative_stats['CreativeId'].astype('float')
# make sure we don't want to merge jobid = 0 
#combined_csv_ss_only = combined_csv_ss_only.groupby(['Job_id', 'creative_id'])['amount'].sum().reset_index()
#combined_csv_ss_creative = combined_csv_ss_only.merge(creative_stats, left_on = ['Job_id', 'creative_id'], right_on = ['JobId', 'CreativeId'], how = 'left')
combined_csv_ss_only['date']  = pd.to_datetime(combined_csv_ss_only['date'] )
combined_csv_ss_only['date'] = combined_csv_ss_only['date'].dt.date
combined_csv_ss_only.loc[combined_csv_ss_only['sub_id'].str.contains( '51797', na = False), 'Shortcode'] = '51797'
combined_csv_ss_only.loc[combined_csv_ss_only['sub_id'].str.contains( '70610',na = False), 'Shortcode'] = '70610'
combined_csv_ss_only.loc[combined_csv_ss_only['sub_id'].str.contains( '44345',na = False), 'Shortcode'] = '44345'
combined_csv_ss_only.loc[combined_csv_ss_only['sub_id'].str.contains( '80837',na = False), 'Shortcode'] = '80837'
print(combined_csv_ss_only['Jump Page Clicks'].sum())

/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_5409/2276811840.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_csv_ss_only_reformatjobs['Job_id'] = combined_csv_ss_only_reformatjobs['sub_id'].str.extract(f'({"|".join(job_ids)})')


6484289.0


In [7]:
""" 
combined_csv_ss_only['Dash_Date_from_subid'] = combined_csv_ss_only['sub_id'].str.extract(r'(\d{1,2}[A-Za-z]{3}\d{2})')
# convert the dateinfo to datetime format using pd.to_datetime()

combined_csv_ss_only.loc[combined_csv_ss_only['first_split'] == 'SS' , 'Dash_Date_from_subid'] = combined_csv_ss_only['split_column'].str[8]
combined_csv_ss_only.loc[combined_csv_ss_only['first_split'] == 'SMS' , 'Dash_Date_from_subid'] = combined_csv_ss_only['split_column'].str[9] 
combined_csv_ss_only.loc[combined_csv_ss_only['Dash_Date_from_subid'] == '1','Dash_Date_from_subid' ] = combined_csv_ss_only['split_column'].str[7] 
combined_csv_ss_only['Dash_Date_from_subid'] = pd.to_datetime(combined_csv_ss_only['Dash_Date_from_subid'], format="%d%b%y", errors='coerce')
"""
combined_csv_ss_only['Hitpath ID'] = combined_csv_ss_only['campaign_id']
#combined_csv_ss_only['Dash_Date_from_subid'] = pd.to_datetime(combined_csv_ss_only['Dash_Date_from_subid'], format="%d%b%y")
# flow data (51797 & 80837 & jobid = 0)
#  by 2023/4/11, we don't include creative id. Subid has that info

flows_clean1 = flows_clean.groupby(['Hitpath ID', 'Date', 'Dataset', 'Shortcode', 'Shortcode Name',\
       'Revenue Source', 'Code_Type', 'AR Flow', 'AR Day', 'AR Flow ID', 'NEW DP.DS or DP.sV','Affiliate_id']).sum().reset_index()
flows_clean1.loc[flows_clean1['AR Day']=='Null', 'AR Day'] = 0 
flows_clean1['AR Day'] = flows_clean1['AR Day'].astype(int)
flows_clean1 = flows_clean1.groupby(['Hitpath ID', 'Date', 'Dataset', 'Shortcode', 'Shortcode Name',
       'Revenue Source', 'Code_Type',  'NEW DP.DS or DP.sV',
       'Affiliate_id']).sum().reset_index()
flows_clean1['AR Day'] = flows_clean1['AR Day'].astype(str)
combined_csv_ss_only.loc[(combined_csv_ss_only['split_column'].str[5].str.contains('AR',na = False)) ,'Hitpath ID'] =combined_csv_ss_only['split_column'].str[6]
combined_csv_ss_ar_flow = combined_csv_ss_only[(combined_csv_ss_only['sub_id'].str.contains('AR')) |  (combined_csv_ss_only['Job_id']==0)]
combined_csv_ss_ar_flow1 = combined_csv_ss_ar_flow.groupby(['affiliate_id','Hitpath ID','Shortcode','Dash_Date_from_subid'])[['amount','Jump Page Clicks']].sum().reset_index()

combined_csv_ss_ar_flow1 = combined_csv_ss_ar_flow1.rename(columns=({'amount': 'Revenue','affiliate_id':'affiliate_id1'}))
flows_clean1 = flows_clean1.groupby(['Hitpath ID', 'Date', 'Dataset', 'Shortcode', 'Shortcode Name',\
       'Revenue Source', 'Code_Type', 'AR Day', \
       'NEW DP.DS or DP.sV', 'Affiliate_id']).sum().reset_index()
print(combined_csv_ss_ar_flow1['Jump Page Clicks'].sum())
flows_clean2 = flows_clean1.merge(combined_csv_ss_ar_flow1, left_on = ['Date','Hitpath ID','Shortcode','Affiliate_id'],right_on = ['Dash_Date_from_subid','Hitpath ID','Shortcode','affiliate_id1'],how = 'outer')
print(flows_clean2['Jump Page Clicks'].sum())
flows_clean2['Send Strategy'] = 'AR'
flows_clean2['Revenue Source'] = 'Short Code - SS Flow'
flows_clean2['Code_Type'] = 'Short Code'
flows_clean2.loc[flows_clean2['Date'].isna(), 'Date'] = flows_clean2['Dash_Date_from_subid']
flows_clean2.loc[(flows_clean2['Shortcode'] == '51797') & (flows_clean2['Shortcode Name'].isna()), 'Shortcode Name'] = 'FLC'
flows_clean2.loc[(flows_clean2['Shortcode'] == '80837') & (flows_clean2['Shortcode Name'].isna()), 'Shortcode Name'] = 'MBC'
flows_clean2['Revenue'] =  flows_clean2['Revenue'].fillna(0)
flows_clean2.columns = flows_clean2.columns.str.title()
#flows_clean2.loc[flows_clean2['Affiliate_Id'].isna(),'Affiliate_Id' ]  = flows_clean2['Affiliate_Id1']
flows_clean_no_delivered = flows_clean2.loc[flows_clean2['Affiliate_Id'].isna(), ] 
flows_clean_no_delivered['Affiliate_Id']= flows_clean_no_delivered['Affiliate_Id1']
flows_clean_no_delivered = flows_clean_no_delivered.dropna(axis =1 , how ='all') 
flows_clean2 = flows_clean2.loc[flows_clean2['Affiliate_Id'].isna() == False, ] 

check_no_match = combined_csv_ss_ar_flow.merge(flows_clean1, how = 'left', right_on = ['Date','Hitpath ID','Shortcode','Affiliate_id'],left_on = ['Dash_Date_from_subid','Hitpath ID','Shortcode','affiliate_id'])
check_no_match1 = check_no_match[check_no_match['Delivered'].isna()]
check_no_match_with_flow = check_no_match1.dropna(axis =1 , how ='all') 


#jobs without creative or job 
combined_csv_ss_only1 = combined_csv_ss_only[~((combined_csv_ss_only['sub_id'].str.contains('AR')) |  (combined_csv_ss_only['Job_id']==0))]
combined_csv_ss_only1 = pd.concat([combined_csv_ss_only1,check_no_match_with_flow]).reset_index(drop = True)
combined_csv_ss_creative_na = combined_csv_ss_only1[(combined_csv_ss_only1['creative_id'].isna())| (combined_csv_ss_only1['creative_id']<100000)| ( (combined_csv_ss_only1['Job_id'].isna()))]

#jobs with creative and job 
combined_csv_ss_creative_notna = combined_csv_ss_only1[~((combined_csv_ss_only1['creative_id'].isna()) | (combined_csv_ss_only1['creative_id']<100000)| ( (combined_csv_ss_only1['Job_id'].isna())))]

# get revenue and delivery stats for jobs with creatives
""" 
combined_csv_ss_creative_notna = combined_csv_ss_creative_notna.groupby(['Job_id', 'creative_id'])['amount'].sum().reset_index()
merge_frame = creative_stats[['JobId', 'CreativeId']].append(combined_csv_ss_creative_notna[['Job_id', 'creative_id']]).drop_duplicates()
creative_stats = creative_stats[creative_stats['Tstamp']>= '2022-11-01']
print(combined_csv_ss_creative_notna['amount'].sum())
combined_csv_ss_creative_notna = combined_csv_ss_creative_notna.merge(creative_stats, left_on = ['Job_id', 'creative_id'], right_on = ['JobId', 'CreativeId'], how = 'left')
print(combined_csv_ss_creative_notna['amount'].sum())
""" 
# get revenue and delivery stats for jobs with creatives
combined_csv_ss_creative_notna = combined_csv_ss_creative_notna.groupby(['Job_id', 'creative_id'])[['amount','Jump Page Clicks']].sum().reset_index()
creative_stats11 = creative_stats[creative_stats['Tstamp']>= '2022-11-01']
creative_stats11 = creative_stats11.rename(columns=({'JobId': 'Job_id', 'CreativeId':'creative_id'}))
creative_stats11['Job_id'] = creative_stats11['Job_id'].astype('int')

merge_frame = pd.concat([creative_stats11[['Job_id', 'creative_id']],combined_csv_ss_creative_notna[['Job_id', 'creative_id']]]).drop_duplicates()
merge_frame = merge_frame.merge(combined_csv_ss_creative_notna, how = 'left')
merge_frame = merge_frame.merge(creative_stats11, how = 'left')

combined_csv_ss_creative_notna = merge_frame.fillna(0)
combined_csv_ss_creative_notna = combined_csv_ss_creative_notna[['Offer','Job_id', 'creative_id', 'CreativeName',\
       'Creative', 'Delivered', 'Cost', 'Optout', 'Clicks', 'amount','Jump Page Clicks']]
combined_csv_ss_creative_notna = combined_csv_ss_creative_notna.rename(columns=({'amount': 'Revenue'}))
#combined_csv_ss_creative_notna = combined_csv_ss_creative_notna.drop_duplicates()
#combined_csv_ss_creative_notna = combined_csv_ss_creative_notna.groupby(['date','Offer','Job_id','CreativeId', 'CreativeName', 'Creative', 'Delivered', 'Cost', 'Unsubcount', 'Clicks']).sum('Revenue').reset_index()

#get jobs information from jobs file
jobs['job_id'] =jobs['job_id'].astype('int')
combined_csv_ss_creative_notna = combined_csv_ss_creative_notna.merge(jobs[['job_id','job_name', 'shortcode', 'start_tstamp',
       'end_tstamp', 'scheduled_tstamp', 'status_text', 'segments','Scheduling Time']], left_on = 'Job_id', right_on = 'job_id', how = 'left')
combined_csv_ss_creative_notna['date'] = pd.to_datetime(combined_csv_ss_creative_notna['scheduled_tstamp']).dt.strftime("%Y-%m-%d")
combined_csv_ss_creative_notna = combined_csv_ss_creative_notna.drop(columns ='job_id')
combined_csv_ss_creative_notna.columns = combined_csv_ss_creative_notna.columns.str.title()
# create send strategy 
combined_csv_ss_creative_notna['Send Strategy'] = np.nan 
combined_csv_ss_creative_notna.loc[ combined_csv_ss_creative_notna['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'P' in l),'Send Strategy'] = 'P'
combined_csv_ss_creative_notna.loc[ combined_csv_ss_creative_notna['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'T' in l),'Send Strategy'] = 'OT'
combined_csv_ss_creative_notna.loc[ combined_csv_ss_creative_notna['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'PT' in l),'Send Strategy'] = 'PT'
combined_csv_ss_creative_notna.loc[ combined_csv_ss_creative_notna['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'AR' in l),'Send Strategy'] = 'AR'
combined_csv_ss_creative_notna.loc[ combined_csv_ss_creative_notna['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'CT' in l),'Send Strategy'] = 'CT'
combined_csv_ss_creative_notna.loc[ combined_csv_ss_creative_notna['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'MI' in l),'Send Strategy'] = 'MI'
combined_csv_ss_creative_notna = combined_csv_ss_creative_notna.reset_index(drop=True)

#get revenue stats for jobs without creatives.
combined_csv_ss_creative_na = combined_csv_ss_creative_na[['date','Job_id', 'campaign_id','amount','sub_id','Jump Page Clicks']]
combined_csv_ss_creative_na = combined_csv_ss_creative_na.rename(columns=({'amount': 'Revenue'}))
#get delivery and click stats for jobs without creatives from job file
 
creative_id_na['Date'] = pd.to_datetime(creative_id_na['Tstamp']).dt.strftime("%Y-%m-%d")
creative_id_na11=creative_id_na[creative_id_na['Date'] >= '2022-11-01']
creative_id_na11['Campaign_Id'] = creative_id_na11['Offer'].str.split(' ',expand = True)[0]
creative_id_na11.loc[creative_id_na11['Campaign_Id'].str.isdigit() == False,'Campaign_Id' ]  = np.nan
creative_id_na11['Campaign_Id']  = creative_id_na11['Campaign_Id'].astype(float)
creative_id_na11['Shortcode Name'] = ''
creative_id_na11.loc[creative_id_na11['Offer'].str.contains('CSS', na = False),'Shortcode Name'] = 'CSS'
creative_id_na11.loc[creative_id_na11['Offer'].str.contains('HZB', na = False),'Shortcode Name'] = 'HZB'
# the jobid didn't use in the combined_csv_ss_creative_notna
#unjoined_creative_stats11 = creative_stats11[~creative_stats11['Job_id'].isin(combined_csv_ss_creative_notna['Job_Id'])]
creative_stats11['Date'] = pd.to_datetime(creative_stats11['Tstamp']).dt.strftime("%Y-%m-%d")
creative_stats11['Campaign_Id'] = creative_stats11['Offer'].str.split(' ',expand = True)[0]
creative_stats11.loc[creative_stats11['Campaign_Id'].str.isdigit() == False,'Campaign_Id' ]  = np.nan
creative_stats11['Campaign_Id']  = creative_stats11['Campaign_Id'].astype(float)
creative_stats11['Shortcode Name'] = ''
creative_stats11.loc[creative_stats11['Offer'].str.contains('CSS', na = False),'Shortcode Name'] = 'CSS'
creative_stats11.loc[creative_stats11['Offer'].str.contains('HZB', na = False),'Shortcode Name'] = 'HZB'

#
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['sub_id'].str.contains('70610' ,na=False ),'shortcode']= '70610'
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['sub_id'].str.contains('44345',na=False),'shortcode']= '44345'
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['sub_id'].str.contains('70610' ,na=False ),'Shortcode Name']= 'CSS'
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['sub_id'].str.contains('44345',na=False),'Shortcode Name']= 'HZB'
combined_csv_ss_creative_na['campaign_id'] = combined_csv_ss_creative_na['sub_id'].str.split('_',expand = True)[6]
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['campaign_id'].str.isdigit()== False, 'campaign_id'] = combined_csv_ss_creative_na['sub_id'].str.split('_',expand = True)[5]
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['campaign_id'].str.isdigit()== False, 'campaign_id'] = combined_csv_ss_creative_na['sub_id'].str.split('_',expand = True)[4]
combined_csv_ss_creative_na['affiliate_id'] = combined_csv_ss_creative_na['sub_id'].str.extract(r'(46\d{4})')
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['campaign_id'].str.isdigit() == False,'campaign_id' ]  = np.nan
combined_csv_ss_creative_na['campaign_id'] = combined_csv_ss_creative_na['campaign_id'].astype(float)
#combined_csv_ss_creative_na['date'] = pd.to_datetime(combined_csv_ss_creative_na['scheduled_tstamp']).dt.strftime("%Y-%m-%d")
job_11 = jobs[jobs['scheduled_tstamp'] >= '2022-11-01']
#creative_stats_limit = creative_stats11[creative_stats11['Job_id'].isin(combined_csv_ss_creative_na['Job_id'].unique().tolist())]


2558332.0
2558332.0


/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_5409/1306772921.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flows_clean_no_delivered['Affiliate_Id']= flows_clean_no_delivered['Affiliate_Id1']
/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_5409/1306772921.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  creative_id_na11['Campaign_Id'] = creative_id_na11['Offer'].str.split(' ',expand = True)[0]
/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_5409/1306772921.py:1

In [8]:
combined_csv_ss_creative_na = combined_csv_ss_creative_na.merge(creative_id_na11[['Date','Scheduling Time','JobId','Offer','Campaign_Id', 'Segment', 'CreativeId', 'CreativeName','Shortcode Name',
       'Creative','Delivered',  'Optout', 'Clicks', 'Cost']], left_on = ['Job_id','campaign_id','date','Shortcode Name'], right_on = ['JobId','Campaign_Id','Date','Shortcode Name'], how = 'outer', copy = False)
combined_csv_ss_creative_na = combined_csv_ss_creative_na.merge(job_11[['job_id','job_name', 'offer', 'shortcode', 'scheduled_tstamp', 'status_text', 'segments', 
        'delivered', 'optout', 'clicks', 'cost']], left_on = ['Job_id'], right_on =['job_id'] , how = 'left' , copy = False)
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['date'].isna(), 'date'] = combined_csv_ss_creative_na['Date']
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['campaign_id'].isna(), 'campaign_id'] = combined_csv_ss_creative_na['Campaign_Id']
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['Offer'].isna(), 'Offer'] = combined_csv_ss_creative_na['offer']
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['Delivered'].isna(), 'Delivered'] = combined_csv_ss_creative_na['delivered']
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['Optout'].isna(), 'Optout'] = combined_csv_ss_creative_na['optout']
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['Clicks'].isna(), 'Clicks'] = combined_csv_ss_creative_na['clicks']
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['Cost'].isna(), 'Cost'] = combined_csv_ss_creative_na['cost']
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['Job_id'].isna(), 'Job_id'] = combined_csv_ss_creative_na['JobId']
combined_csv_ss_creative_na = combined_csv_ss_creative_na.drop(columns = ['job_id','Campaign_Id','Date','offer', 'delivered', 'optout', 'clicks', 'cost','JobId'])
#combined_csv_ss_creative_na = combined_csv_ss_creative_na.reset_index()
#combined_csv_ss_creative_na['date'] = pd.to_datetime(combined_csv_ss_creative_na['scheduled_tstamp']).dt.strftime("%Y-%m-%d")
combined_csv_ss_creative_na.columns = combined_csv_ss_creative_na.columns.str.title()

# create send strategy 
combined_csv_ss_creative_na['Send Strategy'] = np.nan 
combined_csv_ss_creative_na.loc[ combined_csv_ss_creative_na['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'P' in l),'Send Strategy'] = 'P'
combined_csv_ss_creative_na.loc[ combined_csv_ss_creative_na['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'T' in l),'Send Strategy'] = 'T'
combined_csv_ss_creative_na.loc[ combined_csv_ss_creative_na['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'AR' in l),'Send Strategy'] = 'AR'
combined_csv_ss_creative_na.loc[ combined_csv_ss_creative_na['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'CT' in l),'Send Strategy'] = 'CT'
combined_csv_ss_creative_na.loc[ combined_csv_ss_creative_na['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'MI' in l),'Send Strategy'] = 'MI'
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['Job_Id'] == 0, 'Send Strategy'] = 'AR'
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['Job_Id'] == 0, 'Revenue Source'] =  'Short Code - SS Jobs'
#combined_csv_ss_creative_na = combined_csv_ss_creative_na.reset_index(drop=True)


In [9]:
#case #3: revenue for email 
combined_csv_ss_exclude = combined_csv
combined_csv_ss_exclude['affiliate_id'] = combined_csv_ss_exclude['affiliate_id'].astype(str).str.split(".",expand = True)[0]
combined_csv_ss_exclude['site_id'] = combined_csv_ss_exclude['site_id'].astype(str).str.split(".",expand = True)[0]

combined_csv_ss_exclude =  combined_csv_ss_exclude.loc[(combined_csv_ss_exclude['affiliate_id'].isin(email_pubid)) | (combined_csv_ss_exclude['site_id'].isin(email_pubid)),] 
combined_csv_ss_exclude['date'] = pd.to_datetime(combined_csv_ss_exclude['date'])
combined_csv_ss_exclude = combined_csv_ss_exclude[[ 'amount', 'date']]
combined_csv_ss_exclude = combined_csv_ss_exclude.rename(columns=({'amount': 'Revenue','date': 'Date'}))
combined_csv_ss_exclude_11 = combined_csv_ss_exclude[combined_csv_ss_exclude['Date']>='2022-11-01'].reset_index(drop=True)
# case #4: push revenue 
email_pubid_int = emit['Revenue Pub ID'].unique().tolist()
combined_csv_ss_rest =  combined_csv[(combined_csv['first_split']!='SS') & (combined_csv['first_split']!='SMS') & (combined_csv['affiliate_id'].astype(float).isin(email_pubid_int)== False) & (combined_csv['site_id'].astype(float).isin(email_pubid_int)== False) & (combined_csv['first_split']!='MD') &  (combined_csv['first_split']!='TB')  ] 
#combined_csv_ss_rest =  combined_csv[(combined_csv['first_split']!='SS') & (combined_csv['first_split']!='SMS') & (combined_csv['subid_2'].str.contains('FLC|MBC',na = False)== False)& (combined_csv['affiliate_id'].astype(float).isin(email_pubid_int)== False) & (combined_csv['site_id'].astype(float).isin(email_pubid_int)== False) & (combined_csv['first_split']!='MD')   ] 

combined_csv_push = combined_csv_ss_rest.loc[combined_csv_ss_rest['affiliate_name'].str.lower().str.contains("push", na = False),]
combined_csv_push['date'] = pd.to_datetime(combined_csv_push['date'])
combined_csv_push = combined_csv_push[[ 'amount', 'date']]
combined_csv_push = combined_csv_push.rename(columns=({'amount': 'Revenue','date': 'Date'}))
combined_csv_push_11 =  combined_csv_push[combined_csv_push['Date']>='2022-11-01'].reset_index(drop=True)

# case #5: revenue the rest of the revenue. 

combined_csv_ss_last =  combined_csv_ss_rest[ (combined_csv['affiliate_name'].str.lower().str.contains("push", na = False) == False)] 
combined_csv_ss_last_11 = combined_csv_ss_last[combined_csv_ss_last['date']>='2022-11-01'].reset_index(drop=True)

# identify Revenue Sourc
combined_csv_ss_creative_na['Revenue Source'] = 'Short Code - SS Jobs'
combined_csv_ss_creative_notna['Revenue Source'] = 'Short Code - SS Jobs'
combined_csv_ss_exclude['Revenue Source'] = 'Email'
combined_csv_push['Revenue Source'] = 'Push'
historic_data['Revenue Source'] = 'Short Code - SS Jobs'
historic_data['Send Strategy'] = np.nan
#combined_csv_ss_creative_na = combined_csv_ss_creative_na.drop_duplicates()
SS_New_data = pd.concat([combined_csv_ss_creative_notna,combined_csv_ss_creative_na,flows_clean2,combined_csv_ss_exclude, combined_csv_push], axis=0)
SS_New_data['Ecpm'] = SS_New_data['Revenue'] * 1000/SS_New_data['Delivered']
SS_New_data['Roi'] = SS_New_data['Cost'] - SS_New_data['Revenue']
#SS_New_data['Sent'] = SS_New_data['Total']

#combining historic data(before Nov 1) with new data to get SS full data.
#SS_New_data = SS_New_data[historic_data.columns]


SS_Full_data = pd.concat([SS_New_data,historic_data], axis = 0)
SS_Full_data['Date'] = pd.to_datetime(SS_Full_data['Date'])
SS_Full_data = SS_Full_data.sort_values('Date', ascending=False)
SS_Full_data.loc[(SS_Full_data['Job_Id'].isna()) & (SS_Full_data['Revenue Source']==  'Short Code - SS Jobs') ,'Revenue Source'] = 'Short Code - Opt In'
SS_Full_data.loc[ (SS_Full_data['Job_Id'].isna()) & (SS_Full_data['Sub_Id'].str.contains('AR', na = False)) ,'Revenue Source'] =  'Short Code - SS Flow'
SS_Full_data.loc[ (SS_Full_data['Job_Id'].isna())&  (SS_Full_data['Sub_Id'].str.contains('AR', na = False)) ,'Send Strategy'] = 'AR'
SS_Full_data.loc[(SS_Full_data['Job_Id'].isna()) & (SS_Full_data['Revenue Source']==  'Short Code - Opt In') ,'Send Strategy'] = 'Opt In'
SS_Full_data.loc[(SS_Full_data['Job_Id']==0) & (SS_Full_data['Revenue Source']==  'Short Code - SS Jobs') & (SS_Full_data['Send Strategy'].isna()) ,'Send Strategy'] = 'AR'
SS_Full_data.to_csv(localfolder + 'SS_Fulldata.csv', index =False)  

######## SS END ########
  

/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_5409/4057437970.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_csv_ss_exclude['date'] = pd.to_datetime(combined_csv_ss_exclude['date'])
/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_5409/4057437970.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_csv_push['date'] = pd.to_datetime(combined_csv_push['date'])
/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_5409/4057437970.py:24: UserWarning: Boolean Se

In [10]:
# not attribute to our data yet, including TB

combined_csv_ss_last_11['month'] = combined_csv_ss_last_11['date'].astype(str).str[:7]
combined_csv_ss_last_11.to_csv('/Users/liliguo/Desktop/check.csv')
combined_csv_ss_last_11.groupby('month')['amount'].sum()

month
2022-11    4544.49
2022-12    5084.78
2023-01    1560.90
2023-02    1055.98
2023-03     335.37
2023-04     872.45
2023-05     896.70
2023-06      25.75
2023-07       6.35
2023-08     140.34
2023-09       1.70
Name: amount, dtype: float64

In [11]:
combined_csv_ss_last_11

,date,advertiser_name,campaign_id,campaign_name,site_id,affiliate_id,affiliate_name,subid_1,subid_2,subid_3,...,plcategory_name,plsubcategory_name,Jump Page Clicks,subid_1 uc,subid_2 uc,subid_5 uc,sub_id,first_split,Dash_Date_from_subid,month
0,2022-12-05,What If Media,6764.0,Top Credit Card Finder CPC - What If Media (SMS2),460931,460931,1 - SMS LM: Spark Revenue (sweeps),460931,11.14.20_SPK_OCTRAW_TB_LC_TCF_6764,1046007337.0,...,SMS,LM,NaN,0,6,0,11.14.20_SPK_OCTRAW_TB_LC_TCF_6764,11.14.20,2022-12-05,2022-12
1,2022-12-06,What If Media,6764.0,Top Credit Card Finder CPC - What If Media (SMS2),460931,460931,460931_SPK.GFN_SMS,460931,09.25.20_SPK_460931_c_6688_SC_MP,1199142400.0,...,SMS,LM,NaN,0,6,0,09.25.20_SPK_460931_c_6688_SC_MP,09.25.20,2022-12-06,2022-12
2,2022-12-25,Digital Bulldogs / Commission Soup,8612.0,Aspire Cash Back Reward Card - Email Program -...,461247,461247,461247_PA.PS_SMS,461247,461247_80837_MBC_W1,1202364331.0,...,SMS,LM,NaN,0,3,0,1202364331.0,1202364331.0,2022-12-25,2022-12
3,2022-12-23,Spark,11238.0,Inflation Sweeps - SMS ONLY - Spark (SMS),461247,461247,461247_PA.PS_SMS,461247,461247_80837_MBC_W1,1202049251.0,...,SMS,LM,NaN,0,3,0,1202049251.0,1202049251.0,2022-12-23,2022-12
4,2022-12-30,Spark,11224.0,No Fee Credit - SMS ONLY - Spark (SMS),461247,461247,461247_PA.PS_SMS,461247,461247_80837_MBC_W1,1203258336.0,...,SMS,LM,NaN,0,3,0,1203258336.0,1203258336.0,2022-12-30,2022-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5109,2023-07-12,"C3 Data, LLC",NaN,NaN,461313,461313,LM_SMS_NorthPointDirect_RentToOwn,461313,nan,NaN,...,SMS,LM,NaN,0,0,0,NaN,NaN,2023-07-12,2023-07
5110,2023-07-25,Spark,11855.0,Credit America - Spark (SMS),461452,461452,LM_SMS_Comceptual_OpinionShareResearch,461452,test,1237841126,...,SMS,LM,NaN,0,0,0,1237841126,1237841126,2023-07-25,2023-07
5111,2023-07-24,Unlimited Net Resource - Public,5660.0,2023 Credit Card - CPC (PPC + Multisales),460654,460654,I_SMS_RXMG_RentOwnClub,ROC_4.12,nan,NaN,...,SMS,Internal,NaN,1,0,0,NaN,NaN,2023-07-24,2023-07
5112,2023-07-20,RxMG RTO,5786.0,Rent Own Club - CPC - INTERNAL TRANSACTIONS ON...,460654,460654,I_SMS_RXMG_RentOwnClub,ROC_SC_MP_6342_HZB-all-2_w0,nan,NaN,...,SMS,Internal,NaN,5,0,0,ROC_SC_MP_6342_HZB-all-2_w0,ROC,2023-07-20,2023-07


In [12]:
######## MP BEGIN ########
#Read MP file and clean. All stats present in MP_Campaigns file.

MP_campaigns = pd.read_csv(localfolder + 'SMS_SC_MP_Campaigns.csv')
MP_campaigns = MP_campaigns[~MP_campaigns['ACTDATE'].isna()]
MP_campaigns = MP_campaigns[~MP_campaigns['ACTDATE'].isin(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])]
MP_campaigns['ACTDATE']= pd.to_datetime(MP_campaigns['ACTDATE'])
#a = MP_campaigns.isna().sum()
drop_columns = ['Daily Opt Out', 'Unnamed: 28', 'Unnamed: 53', 'Reference', 'c1NEW' , 'Unnamed: 61', 'Unnamed: 62', 'Unnamed: 64', 'Unnamed: 72', 'Unnamed: 73']
MP_campaigns.drop(columns=drop_columns, inplace=True, axis=1)
MP_campaigns= MP_campaigns[~MP_campaigns['Done.'].isna()]
MP_campaigns.columns = MP_campaigns.columns.str.strip()

# remove % and $ symbold
MP_campaigns['REV'] = MP_campaigns['REV'].str.strip()
MP_campaigns['REV'] = MP_campaigns['REV'].replace('$ -', np.nan)
MP_campaigns['REV'] = MP_campaigns['REV'].str.replace('$','')
MP_campaigns['REV'] = MP_campaigns['REV'].str.replace(',','')
MP_campaigns['REV'] = MP_campaigns['REV'].astype('float')

MP_campaigns['COST'] = MP_campaigns['COST'].str.strip()
MP_campaigns['COST'] = MP_campaigns['COST'].replace('$ -', np.nan)
MP_campaigns['COST'] = MP_campaigns['COST'].str.replace('$','')
MP_campaigns['COST'] = MP_campaigns['COST'].astype('float')

MP_campaigns['eCPM'] = MP_campaigns['eCPM'].str.strip()
MP_campaigns['eCPM'] = MP_campaigns['eCPM'].replace('$ -', np.nan)
MP_campaigns['eCPM'] = MP_campaigns['eCPM'].str.replace('$','')
MP_campaigns['eCPM'] = MP_campaigns['eCPM'].astype('float')

MP_campaigns['gPROFIT'] = MP_campaigns['gPROFIT'].str.strip()
MP_campaigns['gPROFIT'] = MP_campaigns['gPROFIT'].str.replace('$','')
MP_campaigns['gPROFIT'] = MP_campaigns['gPROFIT'].replace('-',np.nan)
MP_campaigns['gPROFIT'] = MP_campaigns['gPROFIT'].replace(' -',np.nan)
MP_campaigns['gPROFIT'] = MP_campaigns['gPROFIT'].astype(str).str.replace('\((.*)\)', '-\\1')
MP_campaigns['gPROFIT'] = MP_campaigns['gPROFIT'].str.replace(',','')
MP_campaigns['gPROFIT'] = MP_campaigns['gPROFIT'].astype('float')

MP_campaigns['CLICK %'] = MP_campaigns['CLICK %'].str.strip()
MP_campaigns['CLICK %'] = MP_campaigns['CLICK %'].str.replace('%','')
MP_campaigns['CLICK %'] = MP_campaigns['CLICK %'].astype('float')

MP_campaigns['gMARGIN'] = MP_campaigns['gMARGIN'].str.strip()
MP_campaigns['gMARGIN'] = MP_campaigns['gMARGIN'].str.replace('%','')
MP_campaigns['gMARGIN'] = MP_campaigns['gMARGIN'].astype('float')

MP_campaigns.to_csv(localfolder + 'MP_data.csv', index =False)  

###### MP ENDS #######


/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_5409/3977380537.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  MP_campaigns['ACTDATE']= pd.to_datetime(MP_campaigns['ACTDATE'])


In [44]:
#4 merge SS and MP: not possible as there are columns in MP not in SS.
LC_data = pd.read_csv(localfolder + 'SMS_LC_Campaigns_clean.csv')
SS_data = pd.read_csv(localfolder + 'SS_Fulldata.csv')

lc_df_full = lc_df_full.reset_index(drop=True)
LC_data = LC_data.reset_index(drop=True)
LC_data = pd.concat([lc_df_full,LC_data], axis=0, ignore_index=True)
LC_data.loc[LC_data['Affiliate_Id'].isna(),'Affiliate_Id'] = LC_data['pubID']
LC_data.loc[LC_data['Hitpath_Offer_ID'].isna(),'Hitpath_Offer_ID' ] = LC_data['sid']
LC_data['Affiliate_Id'] = LC_data['Affiliate_Id'].astype(str).str.split('.',expand = True)[0]
LC_data['Hitpath_Offer_ID'] = LC_data['Hitpath_Offer_ID'].astype(str).str.split('.',expand = True)[0]
LC_data['Revenue Source'] = 'Long Code'
SS_data['Code_Type'] = 'Short Code'
LC_data['Code_Type'] = 'Long Code'
SS_data.loc[SS_data['Revenue Source'] == 'Email', 'Code_Type'] = 'Email'
SS_data['Hitpath_Offer_ID'] = SS_data['Offer'].astype('str').str.extract(r'\b(\d{4,5})\b')
SS_data['Affiliate_Id'] = SS_data['Affiliate_Id'].astype(str).str.split('.',expand = True)[0]
#LC_data= LC_data.rename(columns=({'eCPM' : 'Ecpm', 'Clicked': 'Clicks'}))

#merge short code and longcode
merged_data = pd.concat([SS_data,LC_data], axis=0, ignore_index=True)

#merged_data['Hitpath_Offer_ID'] = merged_data['Offer'].astype('str').str.extract(r'\b(\d{4,5})\b')
merged_data.loc[merged_data['Revenue Source']=='Short Code - SS Flow','Hitpath_Offer_ID'] =merged_data['Hitpath Id'].astype('str').str.split('.',expand = True)[0]
merged_data['DP.SV'] = merged_data['New Dp.Ds Or Dp.Sv']
merged_data.loc[merged_data['DP.SV'].isnull(), 'DP.SV'] =  merged_data['Segments'].str.split('_',expand = True)[1]
new_publisher = publisher[['NEW DP.DS or DP.sV','PUBID']]
new_publisher = new_publisher['PUBID'].astype(str).str.split('.',expand = True)[0]
merged_data = merged_data.merge(publisher[['NEW DP.DS or DP.sV','PUBID']], left_on ='DP.SV', right_on = 'NEW DP.DS or DP.sV', how = 'left' )
merged_data.loc[(merged_data['Affiliate_Id'].isna()) |  (merged_data['Affiliate_Id']=='nan'), 'Affiliate_Id'] = merged_data['PUBID']
merged_data['Affiliate_Id'] = merged_data['Affiliate_Id'].astype(str).str.split('.',expand = True)[0]
merged_data['DP&Pub'] = merged_data['DP.SV']+'_'+ merged_data['Affiliate_Id']

merged_data = merged_data[['Date','Scheduling Time', 'Offer','Hitpath_Offer_ID','DP.SV','Affiliate_Id', 'DP&Pub','Job_Id', 'Job_Name','Creative_Id','Creativename','Creative','Send Strategy', 'Shortcode', 'Start_Tstamp', 'Segments', 'Revenue','Jump Page Clicks', 'Delivered', 'Not_Delivered', 'Optout', 'Clicks',
       'Cost', 'Ecpm', 'Time', 'Publisher', 'Campaign', 'Route',  'Carrier', 'Dataset', 'Message',
       'Responder Template', 'Keyword', 'Responder', 'Router Domain Name' , 'c1', 'Responded', 'Response Rate', 'CTR',
        'Gross Profit' , 'Gross Margin', 'RPU' ,'Provider', 'Code_Type','Revenue Source',
     'Ar Day','Sub_Id','Campaign_Id','Roi','Shortcode Name','Total']]

merged_data.loc[merged_data['Shortcode'] == 51797, 'Shortcode Name'] = 'FLC'
merged_data.loc[merged_data['Shortcode'] == 70610, 'Shortcode Name'] = 'CSS'
merged_data.loc[merged_data['Shortcode'] == 44345, 'Shortcode Name'] = 'HZB'
merged_data.loc[merged_data['Shortcode'] == 80837, 'Shortcode Name'] = 'MBC'
merged_data.loc[merged_data['Shortcode'] == 31232, 'Shortcode Name'] = 'DSS'
merged_data.loc[merged_data['Shortcode Name'] == 'FLC', 'Shortcode'] = 51797
merged_data.loc[merged_data['Shortcode Name'] == 'CSS', 'Shortcode'] = 70610
merged_data.loc[merged_data['Shortcode Name'] == 'HZB', 'Shortcode'] = 44345
merged_data.loc[merged_data['Shortcode Name'] == 'MBC', 'Shortcode'] = 80837
merged_data.loc[merged_data['Shortcode Name'] == 'DSS', 'Shortcode'] = 31232
merged_data.loc[merged_data['Delivered'] == '','Delivered'] = np.nan
merged_data['Delivered'] =  merged_data['Delivered'].astype(float)
merged_data.loc[(merged_data['Shortcode Name'] == 'MBC') | (merged_data['Shortcode Name'] == 'FLC'),'Cost']  =  0.00563 *  merged_data['Delivered'] +  merged_data['Cost']
merged_data.loc[(merged_data['Shortcode Name'] == 'UAA') | (merged_data['Shortcode Name'] == 'SVT') | (merged_data['Shortcode Name'] == 'DSS'), 'Cost']  =  0.00545 *  merged_data['Delivered'] 
Offer_name[ 'campaign_id'] = Offer_name[ 'campaign_id'].astype('str')
merged_data = merged_data.merge(Offer_name,left_on = 'Hitpath_Offer_ID',right_on = 'campaign_id',how = 'left')
merged_data['Job_Id']  = merged_data['Job_Id'].astype(str).str.split('.',expand = True)[0].str.strip()
merged_data['Date'] = pd.to_datetime(merged_data['Date'],format ='mixed').dt.strftime("%Y-%m-%d")
merged_data['shortcode_DP.SV'] = merged_data['Shortcode Name'] + "_" + merged_data['DP.SV']
merged_data.loc[merged_data['shortcode_DP.SV']=='_', 'shortcode_DP.SV'] = np.nan
# calculate opportunity cost
merged_data['Opportunity Cost Send Strategy'] =  True
merged_data.loc[merged_data['Send Strategy'].isin(['Null','Opt In',np.nan]), 'Opportunity Cost Send Strategy'] = False
merged_data = merged_data.sort_values('Date')
temp1= merged_data.groupby(['Affiliate_Id','Opportunity Cost Send Strategy','Date']).agg({'Revenue':'sum','Delivered':'sum'}).reset_index()
temp1[['rolling Revenue','rolling Delivered']] = temp1.groupby('Affiliate_Id').shift(1).rolling(30)[['Revenue','Delivered']].sum().reset_index(drop=True)
temp1['Dataset_Agg_30D_eCPM'] = temp1['rolling Revenue'] * 1000/ temp1['rolling Delivered']
dataset_agg_eCPM =  temp1[['Affiliate_Id','Date','Opportunity Cost Send Strategy','Dataset_Agg_30D_eCPM']]
merged_data = merged_data.merge(dataset_agg_eCPM, how = 'left')
merged_data['Opportunity Cost'] = merged_data['Revenue'] - merged_data['Dataset_Agg_30D_eCPM'] * merged_data['Delivered'] /1000 
merged_data = merged_data.sort_values(by = 'Date',ascending = False)
merged_data.replace([np.inf, -np.inf], np.nan, inplace=True)
merged_data.to_csv(localfolder + 'SS_LC_merged_data.csv', index =False)  


/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_5409/6860352.py:3: DtypeWarning: Columns (0,3,4,11,13,14,15,16,17,18,20,23,25,31,32,34,35,38) have mixed types. Specify dtype option on import or set low_memory=False.
  SS_data = pd.read_csv(localfolder + 'SS_Fulldata.csv')


In [45]:
merged_data[merged_data['Segments']== 'DSS_TLG.PL_30DC']['Shortcode Name'].unique()

array(['DSS'], dtype=object)

In [14]:
engagement_daily = merged_data.groupby(['Date','Affiliate_Id'])[['Delivered','Clicks','Jump Page Clicks','Optout','Cost']].sum()
engagement_daily = engagement_daily[~np.all(engagement_daily==0,axis = 1)]
engagement_daily = engagement_daily.reset_index()
daily_revenue_gpby = daily_revenue_gpby.rename(columns = {'amount':'Revenue','affiliate_id': 'Affiliate_Id', 'date': 'Date'})
#Pl_data = daily_revenue_gpby.merge(engagement_daily, left_on =['affiliate_id','date'] ,right_on = ['Affiliate_Id','Date'],copy = False, how = 'outer')
daily_revenue_gpby['Date'] = pd.to_datetime(daily_revenue_gpby['Date'])
engagement_daily['Date'] =  pd.to_datetime(engagement_daily['Date'])
Pl_data = pd.concat([daily_revenue_gpby,engagement_daily], axis=0, ignore_index=True)
Pl_data['PUBID'] =Pl_data['Affiliate_Id'].astype('str').str.split(".",expand = True)[0]
Pl_data = Pl_data.fillna(0)
Pl_data = Pl_data[[ 'Date', 'PUBID','Revenue', 'Delivered', 'Clicks',\
      'Jump Page Clicks', 'Optout','Cost']]
Pl_data = Pl_data.groupby(['PUBID','Date']).sum().reset_index()
#Pl_data.to_csv(localfolder+'p&l_data.csv',index =False)

api_key = pd.read_csv(localfolder+'SMS_SC_SS_Apikey.csv')
publisher1 = publisher
publisher1['NEW DP.DS or DP.sV'] = publisher1['NEW DP.DS or DP.sV'].str.replace('WWM.YFA.2','WWM.YFA')
publisher1['NEW DP.DS or DP.sV'] = publisher1['NEW DP.DS or DP.sV'].str.replace('ZM.PL.2','ZM.PL')
api_key_sms = api_key.merge(publisher1, left_on = 'DP.SV',right_on ='NEW DP.DS or DP.sV',how = 'inner' )
api_key_sms = api_key_sms[['DP.SV','PUBID','Date', 'AcceptedTotal', 'AcceptedNew',\
       'AcceptedDuplicate', 'RejectedTotal', 'RejectedMobile',\
       'RejectedBlacklist', 'RejectedData', 'CostData', 'CostChecks',\
       'CostTotal']]
api_key_sms['Date'] = pd.to_datetime(api_key_sms['Date'])
Pl_data['Date'] =  pd.to_datetime(Pl_data['Date'] )
Pl_data['PUBID']  = Pl_data['PUBID'].astype('str').str.split(".",expand = True)[0]
api_key_sms['PUBID']  = api_key_sms['PUBID'].astype('str').str.split(".",expand = True)[0]
rev_accept_df = pd.concat([Pl_data, api_key_sms], axis=0, ignore_index=True)
rev_accept_df = rev_accept_df.fillna(0)
rev_accept_df['PUBID']  = rev_accept_df['PUBID'].astype('str').str.split(".",expand = True)[0]
rev_accept_df = rev_accept_df.drop(columns = 'DP.SV')
rev_accept_df = rev_accept_df.groupby(['PUBID','Date']).sum().reset_index()


sms_post = pd.read_csv(localfolder + 'SMSposts.csv')
sms_post['Date'] = pd.to_datetime(sms_post['Dates'])
sms_post['PUBID'] = sms_post['Pub Id'].astype('str').str.split(".",expand = True)[0]
sms_post = sms_post.drop(columns = ['Dates','Pub Id','Pub Name'])
post_data = pd.concat([rev_accept_df,sms_post], axis=0, ignore_index=True)
post_data = post_data.fillna(0)
post_data = post_data.groupby(['PUBID','Date']).sum().reset_index()
publisher_raw['PUBID'] = publisher_raw['PUBID'].astype('str').str.split(".",expand = True)[0]
post_data = post_data.merge(publisher_raw[['PUBID', 'NEW PUBLISHER','NEW DP.DS or DP.sV','DMA', 'INTERNAL STATUS',
       'DATA TEAM STATUS', 'Sub Vertical ', 'SCOPE', 'COMPANY (DP)','DP ID']], how = 'left', on = 'PUBID')
post_data.to_csv(localfolder+'p&l_data.csv',index =False)



In [15]:
# offerwall reporting 
offer_sheet = infrastructure.get_smartsheet('offers_sms')
offer_sheet = offer_sheet[offer_sheet['Hitpath ID'].isna() == False]
offer_sheet['Hitpath ID'] = offer_sheet['Hitpath ID'].astype(str).str.split(".",expand = True)[0]
offerwall = offerwall.rename(columns = {'offer_id':'Offer Wall ID'})
print(offerwall[ 'amount'].sum())
offerwall_update = offerwall.merge(offer_sheet[['Hitpath ID','Scheduling Name']], copy = False, how = 'left', left_on = 'Offer Wall ID', right_on = 'Hitpath ID')
print(offerwall_update[ 'amount'].sum())
offerwall_update = offerwall_update.rename(columns = {'Scheduling Name':"Offer Wall Scheduling Name" })
offerwall_update = offerwall_update.merge(offer_sheet[['Hitpath ID','Scheduling Name']], copy = False, how = 'left', left_on = 'campaign_id', right_on = 'Hitpath ID')
offerwall_update.loc[offerwall_update['Scheduling Name'].isna(),'Scheduling Name' ] = offerwall_update['campaign_name']

offerwall_update = offerwall_update.rename(columns = {'Scheduling Name':"Child Offer Campaign Name" })
offerwall_update[['Dash_Date_from_subid','affiliate_id','campaign_id',"Child Offer Campaign Name",'sub_id','Offer Wall ID',"Offer Wall Scheduling Name" ]] = offerwall_update[['Dash_Date_from_subid','affiliate_id','campaign_id',"Child Offer Campaign Name",'sub_id','Offer Wall ID',"Offer Wall Scheduling Name" ]].fillna("NULL")
offerwall_rev = offerwall_update.groupby(['Dash_Date_from_subid','affiliate_id','campaign_id',"Child Offer Campaign Name",'sub_id','Offer Wall ID',"Offer Wall Scheduling Name" ])[[ 'amount', 'Jump Page Clicks']].sum().reset_index()
print(offerwall_rev[ 'amount'].sum())
offerwall_rev['split_column'] = offerwall_rev['sub_id'].str.split('_')
offerwall_rev['Send Strategy'] = 'P'
offerwall_rev.loc[offerwall_rev['split_column'].str[5].str.contains("AR",na = False),'Send Strategy'] = 'AR'
offerwall_rev = offerwall_rev.rename(columns = {'amount':'Child Offer Revenue','Jump Page Clicks':'Child Offer Jump Page Clicks', 'campaign_id':'Child Offer ID'})
merged_data_update = merged_data.add_suffix('- Offer Wall')
offerwall_engage_subid = merged_data_update.groupby(['Sub_Id- Offer Wall'])[['Delivered- Offer Wall','Clicks- Offer Wall','Optout- Offer Wall','Cost- Offer Wall','Revenue- Offer Wall','Jump Page Clicks- Offer Wall']].sum().reset_index()
offerwall_engage =  merged_data_update.groupby(['Date- Offer Wall','Affiliate_Id- Offer Wall','Hitpath_Offer_ID- Offer Wall','Send Strategy- Offer Wall'])[['Delivered- Offer Wall','Clicks- Offer Wall','Optout- Offer Wall','Cost- Offer Wall','Revenue- Offer Wall','Jump Page Clicks- Offer Wall']].sum().reset_index()
offerwall_rev['affiliate_id'] = offerwall_rev['affiliate_id'].astype(str).str.split('.',expand = True)[0]
offerwall_rev['Offer Wall ID'] = offerwall_rev['Offer Wall ID'].astype(str).str.split('.',expand = True)[0]
offerwall_engage['Date- Offer Wall'] = pd.to_datetime(offerwall_engage['Date- Offer Wall'])
offerwall_rev['Dash_Date_from_subid'] = pd.to_datetime(offerwall_rev['Dash_Date_from_subid'])
offerwall_performance = offerwall_rev.merge(offerwall_engage_subid, left_on = ['sub_id'], right_on =['Sub_Id- Offer Wall'], how = 'left' )
offerwall_performance_1 = offerwall_performance.loc[offerwall_performance['Sub_Id- Offer Wall'].isna() == False]
offerwall_performance_2 = offerwall_performance.loc[offerwall_performance['Sub_Id- Offer Wall'].isna()== True].drop(columns = ['Sub_Id- Offer Wall','Delivered- Offer Wall','Clicks- Offer Wall','Optout- Offer Wall','Cost- Offer Wall','Revenue- Offer Wall','Jump Page Clicks- Offer Wall'])
offerwall_performance_2 = offerwall_performance_2.groupby(['Dash_Date_from_subid','affiliate_id','Offer Wall ID','Send Strategy','Child Offer ID'])[[ 'Child Offer Revenue', 'Child Offer Jump Page Clicks']].sum().reset_index()
offerwall_performance_2 = offerwall_performance_2.merge(offerwall_engage, left_on = ['Dash_Date_from_subid','affiliate_id','Offer Wall ID','Send Strategy'], right_on = ['Date- Offer Wall','Affiliate_Id- Offer Wall','Hitpath_Offer_ID- Offer Wall','Send Strategy- Offer Wall'], how ='left')
offerwall_report = pd.concat([offerwall_performance_1,offerwall_performance_2])
print(offerwall_report['Child Offer Revenue'].sum())
offerwall_report.to_csv(localfolder + "offerwall_data.csv")

132437.40225999997
132437.40225999997
132437.40226000003
132437.40226


In [16]:
"""
import os
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
# Replace with your JSON credentials file path and the folder ID where you want to upload the file
CREDENTIALS_FILE = filepath.service_account_location
FOLDER_ID = '1EQWlxXm8lQ5m8uLQPq0czH1-BStjy2a7'

def authenticate_drive():
    creds = service_account.Credentials.from_service_account_file(CREDENTIALS_FILE, scopes=['https://www.googleapis.com/auth/drive'])
    return build('drive', 'v3', credentials=creds)

def upload_file(service, file_path, folder_id):
    file_name = os.path.basename(file_path)
    file_metadata = {
        'name': file_name,
        'parents': folder_id
    }

    media = MediaFileUpload(file_path, resumable=True)

    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()

    print(f"File ID: {file.get('id')}")

if __name__ == '__main__':
    # Replace 'your_file_path' with the path to the file you want to upload
    file_path_to_upload = localfolder + 'SS_LC_merged_data.csv'

    service = authenticate_drive()
    upload_file(service, file_path_to_upload, FOLDER_ID)
""" 

'\nimport os\nfrom google.oauth2 import service_account\nfrom googleapiclient.discovery import build\nfrom googleapiclient.http import MediaFileUpload\n# Replace with your JSON credentials file path and the folder ID where you want to upload the file\nCREDENTIALS_FILE = filepath.service_account_location\nFOLDER_ID = \'1EQWlxXm8lQ5m8uLQPq0czH1-BStjy2a7\'\n\ndef authenticate_drive():\n    creds = service_account.Credentials.from_service_account_file(CREDENTIALS_FILE, scopes=[\'https://www.googleapis.com/auth/drive\'])\n    return build(\'drive\', \'v3\', credentials=creds)\n\ndef upload_file(service, file_path, folder_id):\n    file_name = os.path.basename(file_path)\n    file_metadata = {\n        \'name\': file_name,\n        \'parents\': folder_id\n    }\n\n    media = MediaFileUpload(file_path, resumable=True)\n\n    file = service.files().create(body=file_metadata, media_body=media, fields=\'id\').execute()\n\n    print(f"File ID: {file.get(\'id\')}")\n\nif __name__ == \'__main__\':

## Verification Data Quality 

In [17]:
# jump page clicks 
merged_data.groupby('Date')['Revenue'].sum()

Date
1899-12-31       0.000000
1999-12-31     652.800000
2020-07-27    1681.000000
2020-07-28     775.750000
2020-07-29     924.200000
                 ...     
2023-09-02    4261.200000
2023-09-03    4637.510500
2023-09-04    5589.322495
2023-09-05    5052.458588
2023-09-06     210.550000
Name: Revenue, Length: 1137, dtype: float64

In [18]:
# jump page clicks 
merged_data.groupby('Date')['Jump Page Clicks'].sum()

Date
1899-12-31        0.0
1999-12-31        0.0
2020-07-27        0.0
2020-07-28        0.0
2020-07-29        0.0
               ...   
2023-09-02    11675.0
2023-09-03    12904.0
2023-09-04    13174.0
2023-09-05    12068.0
2023-09-06      442.0
Name: Jump Page Clicks, Length: 1137, dtype: float64

In [19]:
# optout 
merged_data.groupby('Date')['Optout'].sum().tail(20)

Date
2023-08-18    6342.0
2023-08-19    5414.0
2023-08-20    5318.0
2023-08-21    6446.0
2023-08-22    6028.0
2023-08-23    6583.0
2023-08-24    6435.0
2023-08-25    7013.0
2023-08-26    5822.0
2023-08-27    6600.0
2023-08-28    7143.0
2023-08-29    7402.0
2023-08-30    6983.0
2023-08-31    6661.0
2023-09-01    5976.0
2023-09-02    5341.0
2023-09-03    5227.0
2023-09-04    5561.0
2023-09-05    5927.0
2023-09-06       0.0
Name: Optout, dtype: float64

In [20]:
raw_creative_stats

,JobId,Tstamp,Offer,Segment,CreativeId,CreativeName,Creative,DeliverCount,DeliverCost,UnsubCount,...,ClickCount,ConversionCount,Revenue,TotalCost,Net,OOPerc,DeliverClickPerc,ClickConvPerc,ProfitPerc,ECPM
0,0,2021-10-01 00:00:00+00:00,NaN,NaN,NaN,NaN,Welcome to CSS Alerts! Access Your Account: {{...,6734,47.1380,308,...,0,0,0.0,49.29,-49.29,4.6,0.0,0.0,-200,0.0
1,237891,2021-10-01 17:00:00+00:00,B2USRA - Cheris Studio Benefits A,B2 USRA - Day 1,93966.0,DAY 1,CS: Your Membership Request for Assistance has...,0,0.0000,0,...,0,0,0.0,0.00,0.00,0.0,0.0,0.0,-,0.0
2,0,2021-10-02 00:00:00+00:00,NaN,NaN,NaN,NaN,Welcome to CSS Alerts! Access Your Account: {{...,6859,48.0130,277,...,0,0,0.0,49.95,-49.95,4.0,0.0,0.0,-200,0.0
3,238383,2021-10-02 17:00:00+00:00,B2USRA - Cheris Studio Benefits A,B2 USRA - Day 1,93966.0,DAY 1,CS: Your Membership Request for Assistance has...,0,0.0000,0,...,0,0,0.0,0.00,0.00,0.0,0.0,0.0,-,0.0
4,0,2021-10-03 00:00:00+00:00,NaN,NaN,NaN,NaN,Welcome to CSS Alerts! Access Your Account: {{...,6794,47.5580,311,...,0,0,0.0,49.74,-49.74,4.6,0.0,0.0,-200,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31105,592990,2023-09-05 17:00:55+00:00,6322 - BB VAST MBC,MBC_WWM.YFA.2_1PLD.16-30DC_TMO,461620.0,VAST.6322.SC.MBC-P.1,MBC: Your application has been accepted! Vie...,865,1.2975,7,...,55,0,0.0,1.31,-1.31,0.8,6.4,0.0,-200.0,0.0
31106,592966,2023-09-05 17:00:56+00:00,6322 - BB VAST MBC,MBC_PN.SWP_30DC_TMO,461620.0,VAST.6322.SC.MBC-P.1,MBC: Your application has been accepted! Vie...,32406,48.6090,124,...,4285,0,0.0,48.80,-48.80,0.4,13.2,0.0,-200.0,0.0
31107,592968,2023-09-05 17:00:57+00:00,5871 - NC BD MBC,MBC_PN.SWP_30DC_TMO,516714.0,BD.5871.SC.MBC.451671,MBC: You may be eligible for these assistance ...,953,1.4295,1,...,41,0,0.0,1.43,-1.43,0.1,4.3,0.0,-200.0,0.0
31108,593081,2023-09-05 19:00:01+00:00,7878 - DA ADT DSS,DSS_TLG.PL_30DC,516812.0,ADT.7878.SC.DSS.451552,DSS: Your new home security offer comes with a...,931,1.3965,44,...,86,0,0.0,1.46,-1.46,4.7,9.2,0.0,-200.0,0.0


In [21]:
raw_creative_stats['date'] = pd.to_datetime(raw_creative_stats['Tstamp']).dt.strftime("%Y-%m-%d")
raw_creative_stats.groupby('date')['UnsubCount'].sum().tail(20)

date
2023-08-17     4057
2023-08-18     4261
2023-08-19     6507
2023-08-20     5558
2023-08-21    10285
2023-08-22     4946
2023-08-23     8320
2023-08-24     6434
2023-08-25     6187
2023-08-26     4079
2023-08-27     5274
2023-08-28     4643
2023-08-29     3948
2023-08-30     2267
2023-08-31     6028
2023-09-01     4410
2023-09-02     2140
2023-09-03     3634
2023-09-04     4283
2023-09-05     3147
Name: UnsubCount, dtype: int64

In [22]:
flows['date'] = pd.to_datetime(flows['Period'],format = 'mixed').dt.strftime("%Y-%m-%d")
flows.groupby('date')['Optout'].sum().tail(20)

date
2023-08-17    4257
2023-08-18    4275
2023-08-19    3656
2023-08-20    3671
2023-08-21    4231
2023-08-22    4243
2023-08-23    4399
2023-08-24    4387
2023-08-25    4522
2023-08-26    4250
2023-08-27    4101
2023-08-28    4737
2023-08-29    4906
2023-08-30    4785
2023-08-31    4523
2023-09-01    4027
2023-09-02    3556
2023-09-03    3740
2023-09-04    3770
2023-09-05    3980
Name: Optout, dtype: int64

In [23]:
flows.columns

Index(['Id', 'Name', 'Period', 'ListName', 'OfferName',
       'CountAutoresponderWelcome', 'CountAutoresponderHelp',
       'CountAutoresponderStop', 'CountMarketing', 'CountOther', 'Cost',
       'Delivered', 'Clicks', 'CountPreview', 'Optout', 'CountConversion',
       'Revenue', 'Net', 'DeliverClickPerc', 'ClickConvPerc', 'Ecpm',
       'NetEcpm', 'Profit%', 'Revenue Source', 'Code_Type', 'Shortcode Name',
       'Shortcode', 'Dataset', 'Date', 'AR Flow ID', 'AR Day', 'Hitpath ID',
       'AR Flow', 'date'],
      dtype='object')

In [24]:
# Calculate SMS Flow Report
may1_flow = flows_clean[(flows_clean['Date'] >= '2023-04-01') & (flows_clean['Date'] <= '2023-05-02') ]
may1_flow[(may1_flow['Shortcode Name'] =='CSS') &  (may1_flow['Hitpath ID'] == '6363')]['Delivered'].sum()
may1_flow = flows_clean2[(flows_clean2['Date'] >= '2023-04-01') & (flows_clean2['Date'] <= '2023-05-02')]
may1_flow[(may1_flow['Shortcode Name'] =='CSS') &  (may1_flow['Hitpath Id'] == '6363')]['Delivered'].sum()


37890.0

In [25]:
# calculate SMS Offer Report 
combined_csv_ss_creative_na['Offer ID'] = combined_csv_ss_creative_na['Offer'].astype('str').str.extract(r'\b(\d{4,5})\b')
combined_csv_ss_creative_na['Date'] = pd.to_datetime(combined_csv_ss_creative_na['Date'])
may1_creative_na = combined_csv_ss_creative_na[(combined_csv_ss_creative_na['Date'] >= '2023-04-01') & ((combined_csv_ss_creative_na['Date'] <= '2023-05-02'))]
may1_creative_na[may1_creative_na['Offer ID'] == '6363']['Delivered'].sum()

212424.0

In [26]:
# Calculate the difference between Tableau Data & SMS Report
may1 = merged_data[(merged_data['Date'] >= '2023-04-01') & (merged_data['Date'] <= '2023-05-02') ]
may1['Delivered'].sum()
import_df = pd.read_csv('/Users/liliguo/Downloads/sms_report_2023-05-01_2023-05-02.csv')
import_df = pd.read_csv('/Users/liliguo/Downloads/sms_report_2023-04-01_2023-05-03.csv')
import_df
a = may1.groupby(['Shortcode Name','Hitpath_Offer_ID'])['Delivered'].sum().reset_index()
b = import_df.groupby(['Shortcodes','Offer.1'])['Delivered'].sum().reset_index()
c = a.merge(b[['Shortcodes','Offer.1','Delivered']], left_on = ['Shortcode Name','Hitpath_Offer_ID'], right_on = ['Shortcodes','Offer.1'],how = 'left')
c['Delivered_diff'] = c['Delivered_x'] -  c['Delivered_y']
c[(c['Delivered_diff']!=0) & (c['Delivered_diff'].isna() == False)]


,Shortcode Name,Hitpath_Offer_ID,Delivered_x,Shortcodes,Offer.1,Delivered_y,Delivered_diff
29,CSS,6363,250314.0,CSS,6363,318261.0,-67947.0
43,FLC,11244,125126.0,FLC,11244,103070.0,22056.0
44,FLC,11245,116638.0,FLC,11245,95476.0,21162.0
45,FLC,11246,111604.0,FLC,11246,91435.0,20169.0
46,FLC,11247,107278.0,FLC,11247,88099.0,19179.0
49,FLC,11603,99295.0,FLC,11603,82114.0,17181.0
50,FLC,11604,94319.0,FLC,11604,78536.0,15783.0
51,FLC,11605,103486.0,FLC,11605,85195.0,18291.0
52,FLC,11644,140719.0,FLC,11644,117553.0,23166.0
62,HZB,10518,71452.0,HZB,10518,71412.0,40.0


In [27]:
c['Delivered_diff'].sum()

-145394.0

In [28]:
# Check & Examination: 
# Revenue files sum up after November 
rev_file = combined_csv[combined_csv['date']>='2022-11-01'].reset_index(drop=True)['amount'].sum()
# Revenue we made after November
after_merge = combined_csv_ss_creative_na['Revenue'].sum() + combined_csv_ss_creative_notna['Revenue'].sum() + combined_csv_ss_exclude_11['Revenue'].sum() +flows_clean2['Revenue'].sum() + combined_csv_ss_last_11['amount'].sum() + lc_df_full_11['amount'].sum() + combined_csv_push_11['Revenue'].sum()

print("The revenue discrepency after November between raw file and Tableau data is $",round((rev_file - after_merge),2))
print("The total revenue after November is $", round(rev_file,2))
print("The revenue discrepency is", round((rev_file - after_merge) /rev_file,2)*100, "% of the total revenue after November")

The revenue discrepency after November between raw file and Tableau data is $ 1065.45
The total revenue after November is $ 1343092.83
The revenue discrepency is 0.0 % of the total revenue after November


In [29]:
 combined_csv_ss_last_11['amount'].sum()

14524.809999999998

In [30]:
after_merge_cost = combined_csv_ss_creative_na['Cost'].sum() + combined_csv_ss_creative_notna['Cost'].sum()
print("The total Cost after November is $", round(after_merge_cost,2))
print("Margin is", round((rev_file - after_merge_cost) /rev_file,2)*100,"%")

The total Cost after November is $ 389818.49
Margin is 71.0 %


In [31]:
print("Jump Page Clicks discrepancy:",merged_data['Jump Page Clicks'].sum() - jumppageclicks1)

Jump Page Clicks discrepancy: 25821.0


In [32]:
""" 
jobs['date'] = pd.to_datetime(jobs['scheduled_tstamp']).dt.strftime("%Y-%m-%d")
combined_csv_ss_creative_na['Date'] =  pd.to_datetime(combined_csv_ss_creative_na['Date'] ).dt.strftime("%Y-%m-%d")
job_11 = jobs[jobs['scheduled_tstamp'] >= '2022-11-01']
job_11_by_date = job_11.groupby('date')['cost'].sum().reset_index(name = 'cost')
combined_csv_ss_creative_na_cost_only = combined_csv_ss_creative_na.groupby('Date')['Cost'].sum().reset_index(name = 'na_cc_Cost')
combined_csv_ss_creative_na_cost_only['Date'] =  pd.to_datetime(combined_csv_ss_creative_na_cost_only['Date'] ).dt.strftime("%Y-%m-%d")
combined_csv_ss_creative_notna_cost_only = combined_csv_ss_creative_notna.groupby('Date')['Cost'].sum().reset_index(name = 'not_na_cc_Cost')
combined_csv_ss_creative_notna_cost_only['Date'] = pd.to_datetime(combined_csv_ss_creative_notna_cost_only['Date'] ).dt.strftime("%Y-%m-%d")
job_11_by_date1 = job_11_by_date.merge(combined_csv_ss_creative_na_cost_only,left_on = 'date', right_on = 'Date', how = 'left',copy = False)
job_11_by_date2 = job_11_by_date1.merge(combined_csv_ss_creative_notna_cost_only,left_on = 'date', right_on = 'Date', how = 'left', copy = False)
job_11_by_date2['na_cc_Cost'] = job_11_by_date2['na_cc_Cost'].fillna(0)
job_11_by_date2['diff'] = job_11_by_date2['na_cc_Cost'] + job_11_by_date2['not_na_cc_Cost'] - job_11_by_date2['cost']
job_11_by_date2['diff'] = round(job_11_by_date2['diff'],0)
job_11_by_date2[job_11_by_date2['diff']!=0]
"""

' \njobs[\'date\'] = pd.to_datetime(jobs[\'scheduled_tstamp\']).dt.strftime("%Y-%m-%d")\ncombined_csv_ss_creative_na[\'Date\'] =  pd.to_datetime(combined_csv_ss_creative_na[\'Date\'] ).dt.strftime("%Y-%m-%d")\njob_11 = jobs[jobs[\'scheduled_tstamp\'] >= \'2022-11-01\']\njob_11_by_date = job_11.groupby(\'date\')[\'cost\'].sum().reset_index(name = \'cost\')\ncombined_csv_ss_creative_na_cost_only = combined_csv_ss_creative_na.groupby(\'Date\')[\'Cost\'].sum().reset_index(name = \'na_cc_Cost\')\ncombined_csv_ss_creative_na_cost_only[\'Date\'] =  pd.to_datetime(combined_csv_ss_creative_na_cost_only[\'Date\'] ).dt.strftime("%Y-%m-%d")\ncombined_csv_ss_creative_notna_cost_only = combined_csv_ss_creative_notna.groupby(\'Date\')[\'Cost\'].sum().reset_index(name = \'not_na_cc_Cost\')\ncombined_csv_ss_creative_notna_cost_only[\'Date\'] = pd.to_datetime(combined_csv_ss_creative_notna_cost_only[\'Date\'] ).dt.strftime("%Y-%m-%d")\njob_11_by_date1 = job_11_by_date.merge(combined_csv_ss_creative_na_c

In [33]:
job_cost = jobs[jobs['scheduled_tstamp'] >= '2022-11-01']['cost'].sum()
cost_diff = (job_cost - after_merge_cost)
cost_pec = cost_diff / job_cost
print("The cost discrepency after November between raw file and Tableau data is $",round(cost_diff,2))
print("The total revenue after November is $", round(job_cost,2))
print("The cost discrepency is", round(cost_pec,2)*100, "% of the total cost after November")
# normal, because we added kaylera cost      

The cost discrepency after November between raw file and Tableau data is $ -30468.7
The total revenue after November is $ 359349.79
The cost discrepency is -8.0 % of the total cost after November


In [34]:
march = combined_csv_ss_creative_notna[(combined_csv_ss_creative_notna['Date'] >= '2023-03-01') & (combined_csv_ss_creative_notna['Date'] <= '2023-03-31')]
combined_csv_ss_creative_na
march['job0'] = 'NOt 0'
march.loc[march['Job_Id']=='0','job0'] = '0'
march.groupby('job0')['Delivered'].sum()

/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_5409/3293252533.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  march['job0'] = 'NOt 0'


job0
NOt 0    8016657.0
Name: Delivered, dtype: float64

In [35]:
combined_csv_ss_creative_na['Date'] = pd.to_datetime(combined_csv_ss_creative_na['Date'])
march = combined_csv_ss_creative_na[(combined_csv_ss_creative_na['Date'] >= '2023-03-01') & (combined_csv_ss_creative_na['Date'] <= '2023-03-31')]
march['job0'] = 'NOt 0'
march.loc[march['Job_Id']=='0','job0'] = '0'
march.groupby(['job0','Send Strategy'])['Delivered'].sum()


/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_5409/3781066462.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  march['job0'] = 'NOt 0'


job0   Send Strategy
NOt 0  AR               464052.0
Name: Delivered, dtype: float64

In [36]:
a = march.groupby('Job_Id')['Delivered'].sum().reset_index()

In [37]:
temp = pd.read_csv('/Users/liliguo/Downloads/sms_report_2023-03-01_2023-04-01 (1).csv')
b = temp.groupby('JobId')['DeliverCount'].sum().reset_index()

In [38]:
c = a.merge(b, left_on = 'Job_Id',right_on = 'JobId',how = 'outer')
c['Diff'] = c['Delivered'] - c['DeliverCount']
c[c['Diff']!=0]

,Job_Id,Delivered,JobId,DeliverCount,Diff
1,NaN,NaN,499243,600,NaN
2,NaN,NaN,499244,2000,NaN
3,NaN,NaN,499245,4331,NaN
4,NaN,NaN,499246,4070,NaN
5,NaN,NaN,499247,1627,NaN
...,...,...,...,...,...
5485,NaN,NaN,532393,0,NaN
5486,NaN,NaN,532394,2,NaN
5487,NaN,NaN,532395,83,NaN
5488,NaN,NaN,532396,87,NaN


In [39]:
merged_data[merged_data['Date'] == '2023-05-01']['Delivered'].sum()

579747.0